# The main code for computing Wreath Macdonald Polynomials
##    -Marino Romero

In [1]:
FF = QQ['q,t,u,v,w,z'].fraction_field()
q,t = FF.gens()[0],FF.gens()[1]

Sym = SymmetricFunctions(FF)
Sym.inject_shorthands()
Ht = Sym.macdonald().Ht()

## "Residue" needs to be reversed. CAUTION!! The quotient decomposition is reversed on SageMath because of this. 

def residue(self, r, c, l):
        r"""
        Return the ``l``-residue of the cell at row ``r`` and column ``c``.

        The `\ell`-residue of a cell is `c - r` modulo `\ell`.

        This does not strictly depend upon the partition, however, this method
        is included because it is often useful in the context of partitions.

        EXAMPLES::

            sage:Partition([2,1]).residue(1, 0, 3)
                 2
        """
        return (r - c) % l

Partition.residue = residue

## Finding predecessors of a partition according to how many cells of each color are added.

@cached_function
def k_predecessors(mu,k,r):
    r"""
    Return the predecessors of mu by removing k cells of each color 0,...,r-1

    INPUT:

    - ``mu``, ``k``, ``r`` -- partition ``mu``, positive integer ``k``, residue ``r``

    OUTPUT: List of partitions

    EXAMPLES::
        sage: k_predecessors(Partition([6,4,3,2,2]),1,3)
        [[6, 4, 3, 1],
         [6, 3, 3, 1, 1],
         [6, 2, 2, 2, 2],
         [5, 4, 3, 1, 1],
         [4, 4, 3, 2, 1],
         [4, 4, 2, 2, 2]]
    """
    n = mu.size()
    alfa = mu.core(r)
    potential_partitions = list(Partitions(n-r*k, outer=mu))
    return [la for la in potential_partitions if la.core(r) == alfa]

@cached_function
def k_l_predecessors(mu,la,k,r):
    r"""
    Return the partitions nu in between mu and la, with core(nu) = core(mu), where nu is attained from mu
        by removing k cells of each color 0,1,...,r-1.

    INPUT:

    - ``mu``, ``la``, ``k``, ``r`` -- partition ``mu``, partition ``la``, positive integer ``k``, residue ``r``

    OUTPUT: List of partitions

    EXAMPLES::
        sage: k_predecessors(Partition([6,4,3,2,2]),1,3)
        [[6, 4, 3, 1],
         [6, 3, 3, 1, 1],
         [6, 2, 2, 2, 2],
         [5, 4, 3, 1, 1],
         [4, 4, 3, 2, 1],
         [4, 4, 2, 2, 2]]
    """
    n = mu.size()
    alfa = mu.core(r)
    potential_partitions = list(Partitions(n-r*k, outer=mu))
    return [nu for nu in potential_partitions if nu.core(r) == alfa and nu.contains(la)]

@cached_function
def color_s_cells_mu(mu, es, r):
    r"""
    Return the list of cells (a,b) in mu of color s.

    INPUT:

    - ``mu``, ``s``, ``r`` -- partition ``mu``, color ``s``, residue ``r``

    OUTPUT: list of pairs 

    EXAMPLES::

        sage:color_s_cells_mu(Partition([4,2,2]),0,3)
             [(0, 0), (0, 3), (1, 1)]
    """
    mu_cells = mu.cells()
    return [c for c in mu_cells if mod(c[0]-c[1],r) == es]

@cached_function
def color_s_cells_mu_la(mu,la, es, r):
    r"""
    Return the list of cells (a,b) in mu/la of color s.

    INPUT:

    - ``mu``,``la``, ``s``, ``r`` -- partition ``mu``, partition ``la``, color ``s``, residue ``r``

    OUTPUT: list of pairs 

    EXAMPLES::

        sage:color_s_cells_mu_la(Partition([4,2,2]),Partition([1,1]), 0, 3)
             [(0, 3), (1, 1)]
    """
    mu_cells = color_s_cells_mu(mu, es, r)
    la_cells = la.cells()
    return [c for c in mu_cells if c not in la_cells]

@cached_function
def noncore_color_s_cells(mu, es, r):
    r"""
    Return the list of cells (a,b) in mu/core(mu) of color s.

    INPUT:

    - ``mu``, ``s``, ``r`` -- partition ``mu``, color ``s``, residue ``r``

    OUTPUT: an element of the fraction field of polynomials in `q` and `t`

    EXAMPLES::

        sage:noncore_color_s_cells(Partition([4,2,2]), 0, 3)
             [(0, 3), (1, 1)]
    """
    alfa = mu.core(r)
    return color_s_cells_mu_la(mu, alfa, es, r)

## Characters of cells

def char_of_cell(c):
    r"""
    Return the character `t^{c[0]} q^{c[1]}` of the cell .

    INPUT: ``c`` -- a sequence ``c`` of length larger than 1.

    OUTPUT: An element in fraction field of polynomials in `q` and `t`

    EXAMPLES::

        sage:char_of_cell((2,4))
             q^4*t^2
    """
    return t^c[0] * q^c[1]

def cells_to_characters(sells):
    r"""
    Return the list of characters from a list of cells.

    INPUT:

    - ``sells`` -- sequence of pairs ``sells``

    OUTPUT: list of elements in fraction field of polynomials in `q` and `t`

    EXAMPLES::

        sage:cells_to_characters([(0, 0), (0, 3), (1, 1)])
             [1, q^3, q*t]
    """
    return [q^c[1]*t^c[0] for c in sells]

def cells_to_neg_characters(sells):
    r"""
    Return the list of `-q^a t^b` for `(b,a)` in `sells`.

    INPUT:

    - ``list_of_cells`` -- list ``list_of_cells``
    
    OUTPUT: a list of elements of the fraction field of polynomials in `q` and `t`

    EXAMPLES::

        sage:cells_to_neg_characters([(0, 0), (0, 3), (1, 1)])
             [-1, -q^3, -q*t]
    """
    return [-q^c[1]*t^c[0] for c in sells]


def Tmu_s(mu, es, r):
    r"""
    Return the product of `(-q^a t^b)`, where `(b,a)` is a cell in `mu` of color `es` modulo `r`.

    INPUT:

    - ``mu``, ``es``, ``r`` -- partition ``mu``, color ``es``, residue ``r``

    OUTPUT: an element of the fraction field of polynomials in `q` and `t`

    EXAMPLES::

        sage:Tmu_s(Partition([4,2,2]), 0, 3)
             q^4*t
        sage:Tmu_s(Partition([4,2,2]), 1, 3)
             q^3*t^2
    """
    mu_cells_alfa = noncore_color_s_cells(mu,es,r)
    char_list = cells_to_neg_characters(mu_cells_alfa)
    return prod(char_list)

def Tmu_nu_s(mu, nu, es, r):
    r"""
    Return the product of `(-q^a t^b)`, where `(b,a)` is a cell in `mu/nu` of color `es` modulo `r`.

    INPUT:

    - ``mu``, ``s``, ``r`` -- partition ``mu``, partition ``nu``, color ``s``, residue ``r``

    OUTPUT: an element of the fraction field of polynomials in `q` and `t`

    EXAMPLES::

        sage:Tmu_nu_s(Partition([4,4,3,2,2]), Partition([4,2]), 1, 3)
             -q^4*t^7
    """
    mu_cells_nu = color_s_cells_mu_la(mu, nu, es, r)
    char_list = cells_to_neg_characters(mu_cells_nu)
    return prod(char_list)

@cached_function
def B_mu_la_s(mu, la, es, r):
    r"""
    Return the sum of `(q^a t^b)`, where `(a,b)` is a cell in `mu/la` of color `es`.

    INPUT:

    - ``mu``, ``la``, ``es``, ``r`` -- partition ``mu``, partition ``la``, color ``es``, residue ``r``

    OUTPUT: an element of the fraction field of polynomials in `q` and `t`

    EXAMPLES::
        sage:B_mu_la_s(Partition([4,2,2]),Partition([1,1]),0,3)
             q^3 + q*t
    """
    cells_to_add = color_s_cells_mu_la(mu,la, es, r)
    cell_characters = cells_to_characters(cells_to_add)
    return sum(cell_characters)

## Multipartitions/"PartitionTuples" 

def multipartition_to_sequence(gammas,r):
    r"""
    Return a list of pairs `(p,gamma^p_i)` corresponding to a multipartition `(gamma^1,...,gamma^{r-1})`.
    
    INPUT:

    - ``gammas``, ``r`` -- multipartition ``gammas``, length of the list gammas ``r``


    OUTPUT: sequence of pairs

    EXAMPLES::

        sage:multipartition_to_sequence([[2,2],[1]],2)
             [[0, 2], [0, 2], [1, 1]]
    """
    list_of_pairs = []
    for i in range(r):
        list_to_add = [ [i,gammas[i][j]] for j in range(len(gammas[i])) ]
        list_of_pairs = list_of_pairs+list_to_add
    return list_of_pairs

def one_jump(list_of_seqs,k,r):
    r"""
    Return a list of sequences `[la^1,...,la^m]` of partitions by adding one partition to each sequence of parititions 
    such that the last element has k fewer cells of each color than `la^m`.
    
    
    INPUT:

    - ``list_of_seqs``, ``r`` -- a list of sequences of partitions ``list_of_seqs``, residue ``r``


    OUTPUT: a list of sequences of partitions

    EXAMPLES::

        sage:one_jump([[Partition([4,2,2])]],1,3)
             [[[4, 2, 2], [4, 1]], [[4, 2, 2], [3, 2]]]
        sage:one_jump([[Partition([4,2,2])]],2,3)
             [[[4, 2, 2], [1, 1]]]
    """
    new_list = []
    for i in range(len(list_of_seqs)):
        current_seq = list_of_seqs[i]
        la = current_seq[-1]
        new_pars = k_predecessors(la,k,r)
        for nu in new_pars:
            new_term = current_seq + [nu]
            new_list = new_list + [new_term]
    return new_list
        





def sequence_of_partitions_from_jumps(mu,list_of_pairs,r):
    r"""
    Return a list of sequences of partitions `[la^1,...,la^m]` where the jumps from one to the next are given
    by the second coordinates of the list of pairs.
    
    INPUT:

    - ``gammas``, ``list_of_pairs``, ``r`` -- multipartition ``gammas``, list of lists of length 2 ``list_of _pairs``,
                                              number of components of the listed multipartitions ``r``


    OUTPUT: list of sequences of partitions

    EXAMPLES::

        sage:sequence_of_partitions_from_jumps(Partition([4,2,2]),[[1,1],[2,1]],3)
             [[[4, 2, 2], [4, 1], [1, 1]], [[4, 2, 2], [3, 2], [1, 1]]]
    """
    out_list = [[mu]]
    for i in range(len(list_of_pairs)):
        k = list_of_pairs[i][1]
        out_list = one_jump(out_list,k,r)
    return out_list
        

def multipartitions(n, r):
    r"""
    Return all multipartitions of size n with r components.
    
    INPUT:

    - ``n``, ``r`` -- positive integers 


    OUTPUT: MultiPartitions

    EXAMPLES::

        sage:multipartition_to_sequence([[2,2],[1]],2)
             [[0, 2], [0, 2], [1, 1]]
        
    """
    return PartitionTuples(level=r,size=n)


def r_size(la,r):
    return (la.quotient(r)).size()

def color_i_addable_cells(mu,i,r):
    r"""
    Returns all cells of color `i` modulo `r` in `mu`.
    INPUT:
    - ``mu``, ``i``,``r`` -- partition ``mu``, nonnegative integer ``i``, positive integer ``r``
    OUTPUT: MultiPartitions
    EXAMPLES::
        sage:color_i_addable_cells(Partition([4,2,2]),0,3)
             [(3, 0)]
        sage:color_i_addable_cells(Partition([4,2,2]),2,3)
             [(0, 4), (1, 2)]
    """
    return mu.addable_cells_residue(i, r)

def color_i_removable_cells(mu,i,r):
    r"""
    Returns all cells of color `i` modulo `r` in `mu`.
    INPUT:
    - ``mu``, ``i``,``r`` -- partition ``mu``, nonnegative integer ``i``, positive integer ``r``
    OUTPUT: MultiPartitions
    EXAMPLES::
        sage:color_i_addable_cells(Partition([4,2,2]),0,3)
             [(3, 0)]
        sage:color_i_addable_cells(Partition([4,2,2]),2,3)
             [(0, 4), (1, 2)]
    """
    return mu.removable_cells_residue(i, r)

def cells_between_mu_la(mu,la):
    r"""
    Returns cells in `mu` and not in `la`.
    INPUT:
    - ``mu``, ``la`` -- partitions
    OUTPUT: list of pairs
    EXAMPLES::
        sage:cells_between_mu_la(Partition([4,2,2]),Partition([1,1]))
             [(0, 1), (0, 2), (0, 3), (1, 1), (2, 0), (2, 1)]
    """
    return SkewPartition([mu,la]).cells()

def chars_of_cells_between_mu_la(mu,la):
    return cells_to_characters(cells_between_mu_la(mu,la))

def color_of_cell(c,r):
    return mod(c[0]-c[1],r)

@cached_function
def core_to_cees(alfa,r):
    r"""
    


    EXAMPLES: 
        sage:core_to_cees(Partition([1,1]),3)
             [1, 1, -2]
    
    
    """
    Ai = alfa.addable_cells()
    Ri = alfa.removable_cells()
    cees = [0 for i in range(r)]
    cees[0] += 1
    for c in Ai:
        color = color_of_cell(c,r)
        cees[color]+= -1
    for c in Ri:
        color = color_of_cell(c,r)
        cees[color]+=1
    return cees

## Computing c_mu,la, then producing Wreath Macdonald Polynomials from these values.

def total_k(mu, la, r):
    r"""
    Returns the value k giving the number of cells of each color needed to add to la to get mu.

    INPUT:

    - ``mu``, ``la``, ``r`` -- partition ``mu``, partition ``la``, residue ``r``

    OUTPUT: a positive integer

    EXAMPLES::
        sage:total_k(Partition([4,2,2]), Partition([1,1]), 3)
             2
    """
    n = mu.size()
    m = la.size()
    return Integer((n-m)/r)

@cached_function
def cmula_1(mu, la, p, r):
    r"""
    Return the coefficient of `H_la` in `h_1^{\perp}H_\mu`, computing the degree 1 Pieri rule.
    
    INPUT:

    - ``mu``, ``la`` -- partitions where ``mu`` is attained from ``la`` 
                        by adding 1 cell of each color {0,1,...,r-1}
    - ``p``, ``r`` -- color ``p``, residue ``r`` 

    OUTPUT: an element of the fraction field of polynomials in `q` and `t`

    EXAMPLES::
    
        sage:cmula_1(Partition([4,2,2]), Partition([3,2]), 2, 4)
             (q^4*t^6 - q^5*t^3 - q^3*t^5 + q^4*t^2)/(-q^2 + t^2)
        sage:cmula_1(Partition([6,4,4,4,2,2,2]), Partition([6, 4, 3, 3, 2, 2]), 2, 4)
             (q*t^19 - q^6*t^12 - t^18 + q^5*t^11 - q^3*t^13 + q^8*t^6 + q^2*t^12 - q^7*t^5)/(-q^9*t^5 + 2*q^7*t^7 - q^5*t^9 + q^12 - 2*q^10*t^2 + q^8*t^4)
    """
    cells_between = cells_between_mu_la(mu, la)
    list_to_be_mult = []
    for c in cells_between:
        i = color_of_cell(c,r)
        cchar = char_of_cell(c)
        term = 1
        if i == 0:
            term = term * cchar^(-1)
        if i == p:
            term = term*cchar
        Ai = color_i_addable_cells(la,i,r)
        Ri = color_i_removable_cells(la,i,r)
        #if c in Ri:
            #term = term*(q*t-1)
            #print(c)
        for square in Ai:
            term = term * (1 - q*t * cchar/ char_of_cell(square) ) 
        for square in Ri:
            #term = term * cchar^(-1)
            #if square != c:
            term = term/(1-(cchar/char_of_cell(square)))
        list_to_be_mult = list_to_be_mult + [term]
    return prod(list_to_be_mult)/(1- q*t)


@cached_function
def cmula(mu, la, p, es, r):
    r"""
    Return the coefficient of `H_\lambda` in `h_k^{\perp}[X^{(p)}] H_\mu` in terms of the color `es`.
    Part of the Theorem is that this is independent of the choice for `es`.
    
    INPUT:

    - ``mu``, ``la``, ``p``, ``es``, ``r`` -- partition ``mu``, partition ``la``, color ``p``,
                                             color ``es``, residue ``r``


    OUTPUT: an element of the fraction field of polynomials in `q` and `t`

    EXAMPLES::

        sage:cmula(Partition([4,2,2]), Partition([1,1]), 2, 0, 3)
             (-t)/(-q^3)
        sage:cmula(Partition([4,2,2]), Partition([1,1]), 2, 1, 3)
             (-t)/(-q^3)
        sage:cmula(Partition([4,2,2]), Partition([1,1]), 2, 2, 3)
             (-t)/(-q^3)
        
    """
    kl = total_k(mu, la, r)
    k = floor(kl/2)
    if k == 0:
        return cmula_1(mu, la, p, r)
    l = kl-k
    final_sum = 0
    nuus = k_l_predecessors(mu,la,l,r)
    front_factor = e[k](B_mu_la_s(mu,la,es,r)*s[0]).scalar(s[0])
    for nu in nuus:
        final_sum = final_sum + cmula(mu, nu, p, es, r)*cmula(nu, la, p, es, r)*Tmu_nu_s(nu, la, es, r)
    return (-1)^k * final_sum/front_factor

def prod_of_cmula(seeq,ps,es,r):
    r"""
    Return the product of Pieri coefficients `c{seeq[i],seeq[i+1]}^{(p)}`. 
                This should be independent of the choice of ``es``.
    
    INPUT:

    - ``seeq``, ``ps``, ``es``, ``r`` -- sequence of partitions ``seeq``, sequence of colors ``ps``, 
                                         color ``es``, residue ``r``


    OUTPUT: an element of the fraction field of polynomials in `q` and `t`

    EXAMPLES::

        sage:sequence_of_partitions_from_jumps(Partition([4,2,2]),[[1,1],[2,1]],3)
             [[[4, 2, 2], [4, 1], [1, 1]], [[4, 2, 2], [3, 2], [1, 1]]]
    """
    prod_list = [ cmula(seeq[i], seeq[i+1], ps[i], es, r) for i in range(0,len(ps))]
    return prod(prod_list)


def M_mu_Gamma(mu, gammas, es, r):
    r"""
    Return the coefficient of the monomial `m_gammas` in `H_mu`. This should be independent of `es`.
    
    INPUT:

    - ``mu``, ``la``, ``p``, ``es``, ``r`` -- partition ``mu``, multipartition ``gammas``,
                                              color ``es``, residue ``r``


    OUTPUT: an element of the fraction field of polynomials in `q` and `t`

    EXAMPLES::

        sage:multipartition_to_sequence([[2,2],[1]],2)
             [[0, 2], [0, 2], [1, 1]]
        
    """
    final_sum = 0
    list_of_pairs = multipartition_to_sequence(gammas,r)
    all_paths = sequence_of_partitions_from_jumps(mu,list_of_pairs,r)
    ps = [ bee[0] for bee in list_of_pairs]
    for seeq in all_paths:
        final_sum = final_sum + prod_of_cmula(seeq,ps,es,r)
    return final_sum

def multipartition_to_monomial(P):
    r"""
    Return all multipartitions of size n with r components.
    
    INPUT:

    - ``n``, ``r`` -- positive integers 


    OUTPUT: MultiPartitions

    EXAMPLES::

        sage:multipartition_to_sequence([[2,2],[1]],2)
             [[0, 2], [0, 2], [1, 1]]
        
    """
    mon = s(m[P[0]])
    for i in range(1,len(P)):
        mon = mon.tensor(s(m[P[i]]))
    return mon
        
def wH(la,r,es=None):
    r"""
    Return the multi-Schur function expansion of the wreath Macdonald polynomial H_la.
    
    INPUT:

    - ``n``, ``r`` -- positive integers 


    OUTPUT: MultiPartitions

    EXAMPLES::

        sage:multipartition_to_sequence([[2,2],[1]],2)
             [[0, 2], [0, 2], [1, 1]]
        
    """
    laa = Partition(la)
    if es == None:
        es = 0
    n = r_size(laa,r)
    all_multipartitions = multipartitions(n,r)
    HH = 0
    for P in all_multipartitions:
        HH = HH + M_mu_Gamma(laa, P, es, r)*multipartition_to_monomial(P)
    return HH


## Multisymmetric functions. How to expand in different bases. And inner products



def tom(PP):
    r = len(list(PP)[0][0])
    return tensor([m for i in range(r)])(PP)
def tos(PP):
    r = len(list(PP)[0][0])
    return tensor([s for i in range(r)])(PP)
def top(PP):
    r = len(list(PP)[0][0])
    return tensor([p for i in range(r)])(PP)
def toe(PP):
    r = len(list(PP)[0][0])
    return tensor([e for i in range(r)])(PP)
def toh(PP):
    r = len(list(PP)[0][0])
    return tensor([h for i in range(r)])(PP)
def toHt(PP):
    r = len(list(PP)[0][0])
    return tensor([Ht for i in range(r)])(PP)

def find_index_given_first(L,term):
    r"""
    Return the first occurrence of term as the first entry of a sequence in L
    
    INPUT:

    - ``L``, ``term`` --


    OUTPUT:

    EXAMPLES::
        
    """
    l=len(L)
    out = l
    for i in range(l):
        if L[i][0]==term:
            out = i
    return out


def hscalar(f,g):
    r"""
    Return the Hall scalar product of f and g.
    
    INPUT:

    - ``f``, ``g`` --


    OUTPUT:

    EXAMPLES::  
    """ 
    sf = list(tos(f))
    df = PartitionTuple(sf[0][0]).size()
    sg = list(tos(g))
    dg = PartitionTuple(sg[0][0]).size()
    out = 0
    if df == dg:
        for (las,c) in sf:
            d = sg[find_index_given_first(sg,las)][1]
            out = out + c*d
    return out

def qtTerm(r):
    return sum([q^i*t^(i) for i in range(r)])/((1-q^r)*(1-t^r))

def length_of_multipartition(las):
    return sum([len(las[i]) for i in range(len(las))])

def size_of_multipartition(las):
    return (PartitionTuple(las)).size()

def starscalar(f,g):
    r"""
    Return the Star scalar product of f and g.
    
    INPUT:

    - ``f``, ``g`` --


    OUTPUT:

    EXAMPLES::  
    """ 
    pf = list(top(f))
    df = PartitionTuple(pf[0][0]).size()
    pg = list(top(g))
    dg = PartitionTuple(pg[0][0]).size()
    out = 0
    r = len(pf[0][0])
    if df == dg:
        for (las,c) in pf:
            d = pg[find_index_given_first(pg,las)][1]
            out = out + c*d*qtTerm(r)*(-1)^(size_of_multipartition(las)-length_of_multipartition(las))
    return out





def Total_Tmu(mu):
    return prod( [char_of_cell(c) for c in mu.cells()])


class WH(CombinatorialFreeModule):
    def __init__(self,base_ring, r):
        category = Algebras(base_ring.category()).WithBasis()
        category = category.or_subcategory(category)
        CombinatorialFreeModule.__init__(self,base_ring,Partitions(),category = category, prefix = "WH"+str(r))
        self.r = r
        self.Sym = SymmetricFunctions(base_ring)
        
    def _element_constructor_(self, la):
        if la in Partitions():
            return self.monomial(Partition(la))
        elif la in tensor([Sym.s()]*self.r):
            return self.monomial(Partition([]))
            #return sum [ self(par) *coeff(par, la) for par in ]
            

def cleans(f):
    pf = list(tos(f))
    out = 0
    for (las,c) in pf:
        out = out + expand(factor(c))*tensor([s[la] for la in las])
    return out
   # return show(out)

def minusX(f):
    pf = list(top(f))
    r = len(pf[0][0])
    out = 0
    for (las,c) in pf:
        out = out + c*(-1)^(size_of_multipartition(las)-length_of_multipartition(las))*tensor([p[la] for la in las])
    return out

def dagger(f):
    pf = list(top(f))
    r = len(pf[0][0])
    out = 0
    for (las,c) in pf:
        d = expand(factor(expand(c).subs(q=1/q)))
        out = out + d*(-1)^(size_of_multipartition(las)-length_of_multipartition(las))*tensor([p[la] for la in las])
    return out
    



Defining e as shorthand for Symmetric Functions over Fraction Field of Multivariate Polynomial Ring in q, t, u, v, w, z over Rational Field in the elementary basis
Defining f as shorthand for Symmetric Functions over Fraction Field of Multivariate Polynomial Ring in q, t, u, v, w, z over Rational Field in the forgotten basis
Defining h as shorthand for Symmetric Functions over Fraction Field of Multivariate Polynomial Ring in q, t, u, v, w, z over Rational Field in the homogeneous basis
Defining m as shorthand for Symmetric Functions over Fraction Field of Multivariate Polynomial Ring in q, t, u, v, w, z over Rational Field in the monomial basis
Defining p as shorthand for Symmetric Functions over Fraction Field of Multivariate Polynomial Ring in q, t, u, v, w, z over Rational Field in the powersum basis
Defining s as shorthand for Symmetric Functions over Fraction Field of Multivariate Polynomial Ring in q, t, u, v, w, z over Rational Field in the Schur basis


In [2]:
def simpletensor(f,i,r):
    if i == 0:
        out = f
    else: 
        out = p[0]
    for a in range(1,r):
        if a == i:
            out = tensor([out,f])
        else: 
            out = tensor([out,p[0]])
    return out

def modifiedpk(k,i,r):
    out = 0
    for a in range(r):
        for b in range(r):
            out = out + simpletensor(p[k](p[1]*q^a*t^b/((1-q^r)*(1-t^r))),(i+a-b)%r,r)
    return out

def modifiedpla(la,i,r):
    out = 1
    for part in la:
        out = out*modifiedpk(part,i,r)
    return(out)

def modifiedpvecla(las):
    r = len(las)
    out = 1
    for i in range(r):
        out = out * modifiedpla(las[i],i,r)
    return out

def modifiedF(F):
    pf = list(top(f))
    out = 0
    for (las,c) in pf:
        d = expand(factor(c))
        out = out + d*(-1)^(size_of_multipartition(las)-length_of_multipartition(las))*modifiedpvecla(las)
    return out

def starscalar(f,g):
    modf = modifiedF(f)
    return hscalar(modf,g)

In [3]:
%display latex

In [4]:
A = tom(wH([4,4],4))
A

(q^3*t+q^2)*m[] # m[] # m[] # m[1, 1] + q^2*m[] # m[] # m[] # m[2] + (q^4*t+q^3)*m[] # m[] # m[1] # m[1] + (q^5*t+q^4)*m[] # m[] # m[1, 1] # m[] + q^4*m[] # m[] # m[2] # m[] + (q^4+q*t)*m[] # m[1] # m[] # m[1] + (q^5+q^2*t)*m[] # m[1] # m[1] # m[] + (q^3*t+q^2)*m[] # m[1, 1] # m[] # m[] + q^2*m[] # m[2] # m[] # m[] + (q^2*t+q)*m[1] # m[] # m[] # m[1] + (q^3*t+q^2)*m[1] # m[] # m[1] # m[] + (q^3+t)*m[1] # m[1] # m[] # m[] + (q*t+1)*m[1, 1] # m[] # m[] # m[] + m[2] # m[] # m[] # m[]

In [8]:
latex(top(A))

\left(\frac{1}{6} q^{6} t^{3} + \frac{1}{3} q^{5} t^{2} + \frac{1}{3} q^{4} t + \frac{1}{6} q^{3}\right) p_{} \otimes p_{} \otimes p_{} \otimes p_{1,1,1} - \left(\frac{1}{2} q^{6} t^{3} - \frac{1}{2} q^{3}\right) p_{} \otimes p_{} \otimes p_{} \otimes p_{2,1} + \left(\frac{1}{3} q^{6} t^{3} - \frac{1}{3} q^{5} t^{2} - \frac{1}{3} q^{4} t + \frac{1}{3} q^{3}\right) p_{} \otimes p_{} \otimes p_{} \otimes p_{3} + \left(\frac{1}{2} q^{6} t^{2} + q^{5} t + \frac{1}{2} q^{3} t^{3} + \frac{1}{2} q^{4} + \frac{1}{2} q^{2} t^{2}\right) p_{} \otimes p_{} \otimes p_{1} \otimes p_{1,1} - \left(\frac{1}{2} q^{6} t^{2} + \frac{1}{2} q^{3} t^{3} - \frac{1}{2} q^{4} - \frac{1}{2} q^{2} t^{2}\right) p_{} \otimes p_{} \otimes p_{1} \otimes p_{2} + \left(\frac{1}{2} q^{6} t + \frac{1}{2} q^{4} t^{3} + \frac{1}{2} q^{5} + q^{3} t^{2} + \frac{1}{2} q^{2} t\right) p_{} \otimes p_{} \otimes p_{1,1} \otimes p_{1} + \left(\frac{1}{6} q^{5} t^{3} + \frac{1}{3} q^{4} t^{2} + \frac{1}{3} q^{3} t + \frac{1}{6} q^{2}\right) p_{} \otimes p_{} \otimes p_{1,1,1} \otimes p_{} - \left(\frac{1}{2} q^{6} t + \frac{1}{2} q^{4} t^{3} - \frac{1}{2} q^{5} - \frac{1}{2} q^{2} t\right) p_{} \otimes p_{} \otimes p_{2} \otimes p_{1} - \left(\frac{1}{2} q^{5} t^{3} - \frac{1}{2} q^{2}\right) p_{} \otimes p_{} \otimes p_{2,1} \otimes p_{} + \left(\frac{1}{3} q^{5} t^{3} - \frac{1}{3} q^{4} t^{2} - \frac{1}{3} q^{3} t + \frac{1}{3} q^{2}\right) p_{} \otimes p_{} \otimes p_{3} \otimes p_{} + \left(\frac{1}{2} q^{6} t + \frac{1}{2} q^{4} t^{3} + \frac{1}{2} q^{5} + q^{3} t^{2} + \frac{1}{2} q^{2} t\right) p_{} \otimes p_{1} \otimes p_{} \otimes p_{1,1} - \left(\frac{1}{2} q^{6} t + \frac{1}{2} q^{4} t^{3} - \frac{1}{2} q^{5} - \frac{1}{2} q^{2} t\right) p_{} \otimes p_{1} \otimes p_{} \otimes p_{2} + \left(q^{6} + 2 q^{4} t^{2} + 2 q^{3} t + q t^{3}\right) p_{} \otimes p_{1} \otimes p_{1} \otimes p_{1} + \left(\frac{1}{2} q^{5} t^{2} + q^{4} t + \frac{1}{2} q^{2} t^{3} + \frac{1}{2} q^{3} + \frac{1}{2} q t^{2}\right) p_{} \otimes p_{1} \otimes p_{1,1} \otimes p_{} - \left(\frac{1}{2} q^{5} t^{2} + \frac{1}{2} q^{2} t^{3} - \frac{1}{2} q^{3} - \frac{1}{2} q t^{2}\right) p_{} \otimes p_{1} \otimes p_{2} \otimes p_{} + \left(\frac{1}{2} q^{5} t^{2} + q^{4} t + \frac{1}{2} q^{2} t^{3} + \frac{1}{2} q^{3} + \frac{1}{2} q t^{2}\right) p_{} \otimes p_{1,1} \otimes p_{} \otimes p_{1} + \left(\frac{1}{2} q^{5} t + \frac{1}{2} q^{3} t^{3} + \frac{1}{2} q^{4} + q^{2} t^{2} + \frac{1}{2} q t\right) p_{} \otimes p_{1,1} \otimes p_{1} \otimes p_{} + \left(\frac{1}{6} q^{4} t^{3} + \frac{1}{3} q^{3} t^{2} + \frac{1}{3} q^{2} t + \frac{1}{6} q\right) p_{} \otimes p_{1,1,1} \otimes p_{} \otimes p_{} - \left(\frac{1}{2} q^{5} t^{2} + \frac{1}{2} q^{2} t^{3} - \frac{1}{2} q^{3} - \frac{1}{2} q t^{2}\right) p_{} \otimes p_{2} \otimes p_{} \otimes p_{1} - \left(\frac{1}{2} q^{5} t + \frac{1}{2} q^{3} t^{3} - \frac{1}{2} q^{4} - \frac{1}{2} q t\right) p_{} \otimes p_{2} \otimes p_{1} \otimes p_{} - \left(\frac{1}{2} q^{4} t^{3} - \frac{1}{2} q\right) p_{} \otimes p_{2,1} \otimes p_{} \otimes p_{} + \left(\frac{1}{3} q^{4} t^{3} - \frac{1}{3} q^{3} t^{2} - \frac{1}{3} q^{2} t + \frac{1}{3} q\right) p_{} \otimes p_{3} \otimes p_{} \otimes p_{} + \left(\frac{1}{2} q^{5} t^{3} + q^{4} t^{2} + q^{3} t + \frac{1}{2} q^{2}\right) p_{1} \otimes p_{} \otimes p_{} \otimes p_{1,1} - \left(\frac{1}{2} q^{5} t^{3} - \frac{1}{2} q^{2}\right) p_{1} \otimes p_{} \otimes p_{} \otimes p_{2} + \left(q^{5} t^{2} + 2 q^{4} t + q^{2} t^{3} + q^{3} + q t^{2}\right) p_{1} \otimes p_{} \otimes p_{1} \otimes p_{1} + \left(\frac{1}{2} q^{5} t + \frac{1}{2} q^{3} t^{3} + \frac{1}{2} q^{4} + q^{2} t^{2} + \frac{1}{2} q t\right) p_{1} \otimes p_{} \otimes p_{1,1} \otimes p_{} - \left(\frac{1}{2} q^{5} t + \frac{1}{2} q^{3} t^{3} - \frac{1}{2} q^{4} - \frac{1}{2} q t\right) p_{1} \otimes p_{} \otimes p_{2} \otimes p_{} + \left(q^{5} t + q^{3} t^{3} + q^{4} + 2 q^{2} t^{2} + q t\right) p_{1} \otimes p_{1} \otimes p_{} \otimes p_

# 

In [32]:
(Partition([4,3])).pp()

****
***


In [10]:
s[3,2](x1)

NameError: name 'x1' is not defined

In [5]:
A = wH([4,4],4)
cA=cleans(A)
tom(cA)

(q^3*t+q^2)*m[] # m[] # m[] # m[1, 1] + q^2*m[] # m[] # m[] # m[2] + (q^4*t+q^3)*m[] # m[] # m[1] # m[1] + (q^5*t+q^4)*m[] # m[] # m[1, 1] # m[] + q^4*m[] # m[] # m[2] # m[] + (q^4+q*t)*m[] # m[1] # m[] # m[1] + (q^5+q^2*t)*m[] # m[1] # m[1] # m[] + (q^3*t+q^2)*m[] # m[1, 1] # m[] # m[] + q^2*m[] # m[2] # m[] # m[] + (q^2*t+q)*m[1] # m[] # m[] # m[1] + (q^3*t+q^2)*m[1] # m[] # m[1] # m[] + (q^3+t)*m[1] # m[1] # m[] # m[] + (q*t+1)*m[1, 1] # m[] # m[] # m[] + m[2] # m[] # m[] # m[]

In [18]:
A = wH([4,3,1],4)
cA=cleans(A)
latex(tom(cA))

\left(q^{3} t + q^{2}\right) m_{} \otimes m_{} \otimes m_{} \otimes m_{1,1} + q^{2} m_{} \otimes m_{} \otimes m_{} \otimes m_{2} + \left(q^{3} + q t^{2}\right) m_{} \otimes m_{} \otimes m_{1} \otimes m_{1} + \left(q^{2} t^{2} + q t\right) m_{} \otimes m_{} \otimes m_{1,1} \otimes m_{} + q t m_{} \otimes m_{} \otimes m_{2} \otimes m_{} + \left(q^{4} + q t\right) m_{} \otimes m_{1} \otimes m_{} \otimes m_{1} + 2 q^{2} t m_{} \otimes m_{1} \otimes m_{1} \otimes m_{} + \left(q^{3} t + q^{2}\right) m_{} \otimes m_{1,1} \otimes m_{} \otimes m_{} + q^{2} m_{} \otimes m_{2} \otimes m_{} \otimes m_{} + \left(q^{2} t + q\right) m_{1} \otimes m_{} \otimes m_{} \otimes m_{1} + \left(q^{2} + t^{2}\right) m_{1} \otimes m_{} \otimes m_{1} \otimes m_{} + \left(q^{3} + t\right) m_{1} \otimes m_{1} \otimes m_{} \otimes m_{} + \left(q t + 1\right) m_{1,1} \otimes m_{} \otimes m_{} \otimes m_{} + m_{2} \otimes m_{} \otimes m_{} \otimes m_{}

In [7]:
A = wH([4,4,4],4)
cA=cleans(A)
cA

q^6*t^3*s[] # s[] # s[] # s[1, 1, 1] + (q^5*t^2+q^4*t)*s[] # s[] # s[] # s[2, 1] + q^3*s[] # s[] # s[] # s[3] + (q^6*t^2+q^5*t+q^3*t^3)*s[] # s[] # s[1] # s[1, 1] + (q^5*t+q^4+q^2*t^2)*s[] # s[] # s[1] # s[2] + (q^6*t+q^4*t^3+q^3*t^2)*s[] # s[] # s[1, 1] # s[1] + q^5*t^3*s[] # s[] # s[1, 1, 1] # s[] + (q^5+q^3*t^2+q^2*t)*s[] # s[] # s[2] # s[1] + (q^4*t^2+q^3*t)*s[] # s[] # s[2, 1] # s[] + q^2*s[] # s[] # s[3] # s[] + (q^6*t+q^4*t^3+q^3*t^2)*s[] # s[1] # s[] # s[1, 1] + (q^5+q^3*t^2+q^2*t)*s[] # s[1] # s[] # s[2] + (q^6+2*q^4*t^2+2*q^3*t+q*t^3)*s[] # s[1] # s[1] # s[1] + (q^5*t^2+q^4*t+q^2*t^3)*s[] # s[1] # s[1, 1] # s[] + (q^4*t+q^3+q*t^2)*s[] # s[1] # s[2] # s[] + (q^5*t^2+q^4*t+q^2*t^3)*s[] # s[1, 1] # s[] # s[1] + (q^5*t+q^3*t^3+q^2*t^2)*s[] # s[1, 1] # s[1] # s[] + q^4*t^3*s[] # s[1, 1, 1] # s[] # s[] + (q^4*t+q^3+q*t^2)*s[] # s[2] # s[] # s[1] + (q^4+q^2*t^2+q*t)*s[] # s[2] # s[1] # s[] + (q^3*t^2+q^2*t)*s[] # s[2, 1] # s[] # s[] + q*s[] # s[3] # s[] # s[] + (q^5*t^3+q^4*t^2+q^3*t)*s[1] # s[] # s[] # s[1, 1] + (q^4*t^2+q^3*t+q^2)*s[1] # s[] # s[] # s[2] + (q^5*t^2+2*q^4*t+q^2*t^3+q^3+q*t^2)*s[1] # s[] # s[1] # s[1] + (q^5*t+q^3*t^3+q^2*t^2)*s[1] # s[] # s[1, 1] # s[] + (q^4+q^2*t^2+q*t)*s[1] # s[] # s[2] # s[] + (q^5*t+q^3*t^3+q^4+2*q^2*t^2+q*t)*s[1] # s[1] # s[] # s[1] + (q^5+2*q^3*t^2+2*q^2*t+t^3)*s[1] # s[1] # s[1] # s[] + (q^4*t^2+q^3*t+q*t^3)*s[1] # s[1, 1] # s[] # s[] + (q^3*t+q^2+t^2)*s[1] # s[2] # s[] # s[] + (q^4*t^3+q^3*t^2+q^2*t)*s[1, 1] # s[] # s[] # s[1] + (q^4*t^2+q^3*t+q*t^3)*s[1, 1] # s[] # s[1] # s[] + (q^4*t+q^2*t^3+q*t^2)*s[1, 1] # s[1] # s[] # s[] + q^3*t^3*s[1, 1, 1] # s[] # s[] # s[] + (q^3*t^2+q^2*t+q)*s[2] # s[] # s[] # s[1] + (q^3*t+q^2+t^2)*s[2] # s[] # s[1] # s[] + (q^3+q*t^2+t)*s[2] # s[1] # s[] # s[] + (q^2*t^2+q*t)*s[2, 1] # s[] # s[] # s[] + s[3] # s[] # s[] # s[]

In [49]:
A = wH([5,3,3,2],4)
cA=cleans(A)
latex(tom(cA))

\left(\frac{q t^{5} + 2 q^{2} t^{2} + 2 t^{4} + q t}{q^{5}}\right) m_{} \otimes m_{} \otimes m_{} \otimes m_{1,1,1} + \left(\frac{q^{2} t^{2} + t^{4} + q t}{q^{5}}\right) m_{} \otimes m_{} \otimes m_{} \otimes m_{2,1} + \frac{t}{q^{4}} m_{} \otimes m_{} \otimes m_{} \otimes m_{3} + \left(\frac{q^{4} t^{2} + 2 q^{2} t^{4} + t^{6} + q^{3} t + q t^{3}}{q^{6}}\right) m_{} \otimes m_{} \otimes m_{1} \otimes m_{1,1} + \left(\frac{q t^{4} + q^{2} t + t^{3}}{q^{5}}\right) m_{} \otimes m_{} \otimes m_{1} \otimes m_{2} + \left(\frac{q^{3} t^{4} + q t^{6} + q^{4} t + 2 q^{2} t^{3} + t^{5}}{q^{6}}\right) m_{} \otimes m_{} \otimes m_{1,1} \otimes m_{1} + \left(\frac{q t^{6} + 2 q^{2} t^{3} + 2 t^{5} + q t^{2}}{q^{5}}\right) m_{} \otimes m_{} \otimes m_{1,1,1} \otimes m_{} + \left(\frac{q^{4} t + q^{2} t^{3} + t^{5}}{q^{6}}\right) m_{} \otimes m_{} \otimes m_{2} \otimes m_{1} + \left(\frac{q^{2} t^{3} + t^{5} + q t^{2}}{q^{5}}\right) m_{} \otimes m_{} \otimes m_{2,1} \otimes m_{} + \frac{t^{2}}{q^{4}} m_{} \otimes m_{} \otimes m_{3} \otimes m_{} + \left(\frac{q^{5} t^{2} + q^{4} t + 2 q^{2} t^{3} + t^{5} + q t^{2}}{q^{6}}\right) m_{} \otimes m_{1} \otimes m_{} \otimes m_{1,1} + \left(\frac{q^{3} t + q t^{3} + t^{2}}{q^{5}}\right) m_{} \otimes m_{1} \otimes m_{} \otimes m_{2} + \left(\frac{q^{5} t + 2 q^{3} t^{3} + q t^{5} + q^{2} t^{2} + t^{4}}{q^{6}}\right) m_{} \otimes m_{1} \otimes m_{1} \otimes m_{1} + \left(\frac{q^{3} t^{3} + q t^{5} + 2 q^{2} t^{2} + 2 t^{4}}{q^{5}}\right) m_{} \otimes m_{1} \otimes m_{1,1} \otimes m_{} + \left(\frac{2 q^{2} t^{2} + t^{4}}{q^{5}}\right) m_{} \otimes m_{1} \otimes m_{2} \otimes m_{} + \left(\frac{q^{4} t^{3} + q^{5} + 2 q^{3} t^{2} + q t^{4} + t^{3}}{q^{6}}\right) m_{} \otimes m_{1,1} \otimes m_{} \otimes m_{1} + \left(\frac{2 q^{3} t^{2} + 2 q t^{4} + q^{2} t + t^{3}}{q^{5}}\right) m_{} \otimes m_{1,1} \otimes m_{1} \otimes m_{} + \left(\frac{q t^{4} + 2 q^{2} t + 2 t^{3} + q}{q^{4}}\right) m_{} \otimes m_{1,1,1} \otimes m_{} \otimes m_{} + \left(\frac{q^{5} + q^{3} t^{2} + t^{3}}{q^{6}}\right) m_{} \otimes m_{2} \otimes m_{} \otimes m_{1} + \left(\frac{q^{3} t^{2} + q^{2} t + t^{3}}{q^{5}}\right) m_{} \otimes m_{2} \otimes m_{1} \otimes m_{} + \left(\frac{q^{2} t + t^{3} + q}{q^{4}}\right) m_{} \otimes m_{2,1} \otimes m_{} \otimes m_{} + \frac{1}{q^{3}} m_{} \otimes m_{3} \otimes m_{} \otimes m_{} + \left(\frac{q^{5} t^{2} + q^{4} t + 2 q^{2} t^{3} + t^{5} + q t^{2}}{q^{5}}\right) m_{1} \otimes m_{} \otimes m_{} \otimes m_{1,1} + \left(\frac{q^{3} t + q t^{3} + t^{2}}{q^{4}}\right) m_{1} \otimes m_{} \otimes m_{} \otimes m_{2} + \left(\frac{q^{5} t + 2 q^{3} t^{3} + q t^{5} + q^{2} t^{2} + t^{4}}{q^{5}}\right) m_{1} \otimes m_{} \otimes m_{1} \otimes m_{1} + \left(\frac{q^{3} t^{3} + q t^{5} + 2 q^{2} t^{2} + 2 t^{4}}{q^{4}}\right) m_{1} \otimes m_{} \otimes m_{1,1} \otimes m_{} + \left(\frac{2 q^{2} t^{2} + t^{4}}{q^{4}}\right) m_{1} \otimes m_{} \otimes m_{2} \otimes m_{} + \left(\frac{q^{4} t^{3} + q^{5} + 2 q^{3} t^{2} + q t^{4} + t^{3}}{q^{5}}\right) m_{1} \otimes m_{1} \otimes m_{} \otimes m_{1} + \left(\frac{2 q^{3} t^{2} + 2 q t^{4} + q^{2} t + t^{3}}{q^{4}}\right) m_{1} \otimes m_{1} \otimes m_{1} \otimes m_{} + \left(\frac{q t^{4} + 2 q^{2} t + 2 t^{3} + q}{q^{3}}\right) m_{1} \otimes m_{1,1} \otimes m_{} \otimes m_{} + \left(\frac{q^{2} t + t^{3} + q}{q^{3}}\right) m_{1} \otimes m_{2} \otimes m_{} \otimes m_{} + \left(\frac{q^{4} t^{3} + q^{5} + 2 q^{3} t^{2} + q t^{4} + t^{3}}{q^{4}}\right) m_{1,1} \otimes m_{} \otimes m_{} \otimes m_{1} + \left(\frac{2 q^{3} t^{2} + 2 q t^{4} + q^{2} t + t^{3}}{q^{3}}\right) m_{1,1} \otimes m_{} \otimes m_{1} \otimes m_{} + \left(\frac{q t^{4} + 2 q^{2} t + 2 t^{3} + q}{q^{2}}\right) m_{1,1} \otimes m_{1} \otimes m_{} \otimes m_{} + \left(\frac{q t^{4} + 2 q^{2} t + 2 t^{3} + q}{q}\right) m_{1,1,1} \otimes m_{} \otimes m_{} \otimes m_{} + \left(\frac{q^{5} + q^{3} t^{2} + t^{3}}{q^{4}}\right) m_{2} \otimes m_{} \otimes m_{} \otimes m_{1} + \left(\frac{q

In [56]:
Partition([5,3,3,2]).quotient(4)

([1, 1], [1], [], [])

In [50]:
A = wH([6,4,3],4)
cA=cleans(A)
latex(tom(cA))

\left(\frac{q^{6} t + 2 q^{5} + 2 q^{2} t + q}{t^{2}}\right) m_{} \otimes m_{} \otimes m_{} \otimes m_{1,1,1} + \left(\frac{q^{5} + q^{2} t + q}{t^{2}}\right) m_{} \otimes m_{} \otimes m_{} \otimes m_{2,1} + \frac{q}{t^{2}} m_{} \otimes m_{} \otimes m_{} \otimes m_{3} + \left(\frac{q^{6} + 2 q^{3} t + q t^{3} + q^{2} + t^{2}}{t^{2}}\right) m_{} \otimes m_{} \otimes m_{1} \otimes m_{1,1} + \left(\frac{q^{3} t + q^{2} + t^{2}}{t^{2}}\right) m_{} \otimes m_{} \otimes m_{1} \otimes m_{2} + \left(\frac{q^{6} t + q^{4} t^{3} + q^{5} + 2 q^{3} t^{2} + t^{3}}{q^{2} t^{2}}\right) m_{} \otimes m_{} \otimes m_{1,1} \otimes m_{1} + \left(\frac{q^{5} t + 2 q^{4} + 2 q t + 1}{q^{2}}\right) m_{} \otimes m_{} \otimes m_{1,1,1} \otimes m_{} + \left(\frac{q^{5} + q^{3} t^{2} + t^{3}}{q^{2} t^{2}}\right) m_{} \otimes m_{} \otimes m_{2} \otimes m_{1} + \left(\frac{q^{4} + q t + 1}{q^{2}}\right) m_{} \otimes m_{} \otimes m_{2,1} \otimes m_{} + \frac{1}{q^{2}} m_{} \otimes m_{} \otimes m_{3} \otimes m_{} + \left(\frac{2 q^{4} t + q^{3} + 2 q t^{2} + t}{t^{2}}\right) m_{} \otimes m_{1} \otimes m_{} \otimes m_{1,1} + \left(\frac{q^{3} + q t^{2} + t}{t^{2}}\right) m_{} \otimes m_{1} \otimes m_{} \otimes m_{2} + \left(\frac{q^{5} + 2 q^{3} t^{2} + q^{2} t + 2 t^{3}}{q t^{2}}\right) m_{} \otimes m_{1} \otimes m_{1} \otimes m_{1} + \left(\frac{q^{6} t + q^{5} + 2 q^{3} t^{2} + q^{2} t + t^{3}}{q^{3} t}\right) m_{} \otimes m_{1} \otimes m_{1,1} \otimes m_{} + \left(\frac{q^{5} + q^{2} t + t^{3}}{q^{3} t}\right) m_{} \otimes m_{1} \otimes m_{2} \otimes m_{} + \left(\frac{q^{4} t + 2 q^{3} + q t^{2} + 2 t}{q t}\right) m_{} \otimes m_{1,1} \otimes m_{} \otimes m_{1} + \left(\frac{q^{6} + q^{4} t^{2} + 2 q^{3} t + q t^{3} + t^{2}}{q^{3} t}\right) m_{} \otimes m_{1,1} \otimes m_{1} \otimes m_{} + \left(\frac{q^{5} t + 2 q^{4} + 2 q t + 1}{q^{3}}\right) m_{} \otimes m_{1,1,1} \otimes m_{} \otimes m_{} + \left(\frac{q^{3} + 2 t}{q t}\right) m_{} \otimes m_{2} \otimes m_{} \otimes m_{1} + \left(\frac{2 q^{3} + t}{q^{3}}\right) m_{} \otimes m_{2} \otimes m_{1} \otimes m_{} + \left(\frac{q^{4} + q t + 1}{q^{3}}\right) m_{} \otimes m_{2,1} \otimes m_{} \otimes m_{} + \frac{1}{q^{3}} m_{} \otimes m_{3} \otimes m_{} \otimes m_{} + \left(\frac{2 q^{5} t + q^{4} + 2 q^{2} t^{2} + q t}{t^{2}}\right) m_{1} \otimes m_{} \otimes m_{} \otimes m_{1,1} + \left(\frac{q^{4} + q^{2} t^{2} + q t}{t^{2}}\right) m_{1} \otimes m_{} \otimes m_{} \otimes m_{2} + \left(\frac{q^{5} + 2 q^{3} t^{2} + q^{2} t + 2 t^{3}}{t^{2}}\right) m_{1} \otimes m_{} \otimes m_{1} \otimes m_{1} + \left(\frac{q^{6} t + q^{5} + 2 q^{3} t^{2} + q^{2} t + t^{3}}{q^{2} t}\right) m_{1} \otimes m_{} \otimes m_{1,1} \otimes m_{} + \left(\frac{q^{5} + q^{2} t + t^{3}}{q^{2} t}\right) m_{1} \otimes m_{} \otimes m_{2} \otimes m_{} + \left(\frac{q^{4} t + 2 q^{3} + q t^{2} + 2 t}{t}\right) m_{1} \otimes m_{1} \otimes m_{} \otimes m_{1} + \left(\frac{q^{6} + q^{4} t^{2} + 2 q^{3} t + q t^{3} + t^{2}}{q^{2} t}\right) m_{1} \otimes m_{1} \otimes m_{1} \otimes m_{} + \left(\frac{q^{5} t + 2 q^{4} + 2 q t + 1}{q^{2}}\right) m_{1} \otimes m_{1,1} \otimes m_{} \otimes m_{} + \left(\frac{q^{4} + q t + 1}{q^{2}}\right) m_{1} \otimes m_{2} \otimes m_{} \otimes m_{} + \left(\frac{q^{5} t + 2 q^{4} + q^{2} t^{2} + 2 q t}{t}\right) m_{1,1} \otimes m_{} \otimes m_{} \otimes m_{1} + \left(\frac{q^{6} + q^{4} t^{2} + 2 q^{3} t + q t^{3} + t^{2}}{q t}\right) m_{1,1} \otimes m_{} \otimes m_{1} \otimes m_{} + \left(\frac{q^{5} t + 2 q^{4} + 2 q t + 1}{q}\right) m_{1,1} \otimes m_{1} \otimes m_{} \otimes m_{} + \left(q^{5} t + 2 q^{4} + 2 q t + 1\right) m_{1,1,1} \otimes m_{} \otimes m_{} \otimes m_{} + \left(\frac{q^{4} + 2 q t}{t}\right) m_{2} \otimes m_{} \otimes m_{} \otimes m_{1} + \left(\frac{2 q^{3} + t}{q}\right) m_{2} \otimes m_{} \otimes m_{1} \otimes m_{} + \left(\frac{q^{4} + q t + 1}{q}\right) m_{2} \otimes m_{1} \otimes m_{} \otimes m_{} + \left(q^{4} + q t + 1\right) m_{2,1} \otimes m_{} \otimes m_{} \otimes m_{} + m_{3}

In [10]:
A = wH([3,2,1,1,1,1,1],2)
cA=cleans(A)
latex(tom(cA))

\left(\frac{t^{2} + 1}{t^{2}}\right) m_{} \otimes m_{1,1} + \frac{1}{t^{2}} m_{} \otimes m_{2} + \left(\frac{t^{2} + 1}{t}\right) m_{1} \otimes m_{1} + \left(t^{2} + 1\right) m_{1,1} \otimes m_{} + m_{2} \otimes m_{}

In [4]:
A = wH([7,2,1],2)
cA=cleans(A)
latex(cA)

s_{} \otimes s_{1,1} + \frac{1}{q^{2}} s_{} \otimes s_{2} + \left(\frac{q^{2} + 1}{q}\right) s_{1} \otimes s_{1} + q^{2} s_{1,1} \otimes s_{} + s_{2} \otimes s_{}

In [85]:
A = wH([5,2,1,1,1],2)
cA=cleans(A)
latex(cA)

\frac{1}{q t} s_{} \otimes s_{1,1} + \frac{1}{q t} s_{} \otimes s_{2} + \left(\frac{q + t}{q t}\right) s_{1} \otimes s_{1} + s_{1,1} \otimes s_{} + s_{2} \otimes s_{}

In [95]:
A = wH([3,2,2,1],2)
cA=cleans(A)
latex(cA)

q^{2} t^{6} s_{} \otimes s_{1,1,1,1} + \left(q^{2} t^{4} + q t^{5} + t^{6}\right) s_{} \otimes s_{2,1,1} + \left(\frac{t^{7} + q^{3} t^{2}}{q}\right) s_{} \otimes s_{2,2} + \left(\frac{q^{2} t^{3} + q t^{4} + t^{5}}{q}\right) s_{} \otimes s_{3,1} + \frac{t^{3}}{q} s_{} \otimes s_{4} + \left(q^{2} t^{5} + q t^{6} + q^{2} t^{3} + q t^{4}\right) s_{1} \otimes s_{1,1,1} + \left(\frac{q^{3} t^{3} + 2 q^{2} t^{4} + 2 q t^{5} + t^{6} + q^{3} t + q^{2} t^{2}}{q}\right) s_{1} \otimes s_{2,1} + \left(\frac{q^{2} t^{2} + 2 q t^{3} + t^{4}}{q}\right) s_{1} \otimes s_{3} + \left(q^{2} t^{4} + q t^{5} + t^{6} + q^{3} t + q^{2} t^{2} + q t^{3}\right) s_{1,1} \otimes s_{1,1} + \left(2 q^{2} t^{2} + 2 q t^{3} + 2 t^{4}\right) s_{1,1} \otimes s_{2} + \left(q^{3} t^{2} + 2 q^{2} t^{3} + q t^{4}\right) s_{1,1,1} \otimes s_{1} + q^{3} t^{3} s_{1,1,1,1} \otimes s_{} + \left(q t^{5} + q^{2} t^{2} + 2 q t^{3} + t^{4} + q t\right) s_{2} \otimes s_{1,1} + \left(\frac{q^{2} t^{3} + q t^{4} + t^{5} + q^{3} + q^{2

In [87]:
A = wH([5,3,2],2)
cA=cleans(A)
latex(cA)

q^{7} t^{4} s_{} \otimes s_{1,1,1,1,1} + \left(q^{7} t^{2} + q^{6} t^{3} + q^{5} t^{4} + q^{3} t^{4}\right) s_{} \otimes s_{2,1,1,1} + \left(2 q^{5} t^{2} + 2 q^{4} t^{3} + q^{3} t^{4}\right) s_{} \otimes s_{2,2,1} + \left(q^{6} t + q^{5} t^{2} + q^{4} t^{3} + q^{3} t^{2} + q^{2} t^{3} + q t^{4}\right) s_{} \otimes s_{3,1,1} + \left(q^{4} t + 2 q^{3} t^{2} + 2 q^{2} t^{3}\right) s_{} \otimes s_{3,2} + \left(q^{4} t + q^{2} t + q t^{2} + t^{3}\right) s_{} \otimes s_{4,1} + t s_{} \otimes s_{5} + \left(q^{7} t^{3} + q^{6} t^{4} + q^{6} t^{2} + q^{5} t^{3} + q^{4} t^{4}\right) s_{1} \otimes s_{1,1,1,1} + \left(q^{7} t + 2 q^{6} t^{2} + 2 q^{5} t^{3} + q^{4} t^{4} + q^{5} t + 3 q^{4} t^{2} + 3 q^{3} t^{3} + 2 q^{2} t^{4}\right) s_{1} \otimes s_{2,1,1} + \left(2 q^{5} t + 3 q^{4} t^{2} + 3 q^{3} t^{3} + q^{2} t^{4} + q^{2} t^{2}\right) s_{1} \otimes s_{2,2} + \left(q^{6} + 2 q^{5} t + 2 q^{4} t^{2} + q^{3} t^{3} + 2 q^{3} t + 3 q^{2} t^{2} + 3 q t^{3} + t^{4}\right) s_{1} \otimes s_{3,1} + 

In [96]:
A = wH([3,2,2,2,1],2)
cA=cleans(A)
latex(cA)

\frac{q}{t^{3}} s_{} \otimes s_{1,1} + \frac{1}{t^{2}} s_{} \otimes s_{2} + \left(\frac{q + t}{t^{2}}\right) s_{1} \otimes s_{1} + \frac{q}{t} s_{1,1} \otimes s_{} + s_{2} \otimes s_{}

In [81]:
A = wH([3,2,1],2)
cA=cleans(A)
latex(cA)

s_{} \otimes s_{}

In [15]:
A = wH([3,2,2],2)
cA=cleans(A)
latex(cA)

t^{3} s_{} \otimes s_{1,1,1} + \left(\frac{q t + t^{2}}{q}\right) s_{} \otimes s_{2,1} + \frac{1}{q} s_{} \otimes s_{3} + \left(\frac{q^{2} t + q t^{2} + t^{3}}{q}\right) s_{1} \otimes s_{1,1} + \left(\frac{q t^{2} + q + t}{q}\right) s_{1} \otimes s_{2} + \left(q t^{2} + t^{3} + t\right) s_{1,1} \otimes s_{1} + q t^{3} s_{1,1,1} \otimes s_{} + \left(\frac{q^{2} + q t + t^{2}}{q}\right) s_{2} \otimes s_{1} + \left(q t + t^{2}\right) s_{2,1} \otimes s_{} + s_{3} \otimes s_{}

In [17]:
A = wH([3,3,2],2)
cA=cleans(A)
latex(cA)

q^{4} t^{4} s_{} \otimes s_{1,1,1,1} + \left(q^{4} t^{2} + q^{3} t^{3} + q^{2} t^{4}\right) s_{} \otimes s_{2,1,1} + \left(q^{3} t^{3} + q^{2} t^{2}\right) s_{} \otimes s_{2,2} + \left(q^{3} t + q^{2} t^{2} + q t^{3}\right) s_{} \otimes s_{3,1} + q t s_{} \otimes s_{4} + \left(q^{4} t^{3} + q^{3} t^{4} + q^{3} t^{2} + q^{2} t^{3}\right) s_{1} \otimes s_{1,1,1} + \left(q^{4} t + 2 q^{3} t^{2} + 2 q^{2} t^{3} + q t^{4} + q^{2} t + q t^{2}\right) s_{1} \otimes s_{2,1} + \left(q^{3} + q^{2} t + q t^{2} + t^{3}\right) s_{1} \otimes s_{3} + \left(q^{4} t^{2} + q^{3} t^{3} + q^{2} t^{4} + q^{3} t + q^{2} t^{2} + q t^{3}\right) s_{1,1} \otimes s_{1,1} + \left(q^{4} + q^{3} t + 2 q^{2} t^{2} + q t^{3} + t^{4}\right) s_{1,1} \otimes s_{2} + \left(q^{4} t + q^{3} t^{2} + q^{2} t^{3} + q t^{4}\right) s_{1,1,1} \otimes s_{1} + q^{3} t^{3} s_{1,1,1,1} \otimes s_{} + \left(q^{3} t^{3} + q^{3} t + 2 q^{2} t^{2} + q t^{3} + q t\right) s_{2} \otimes s_{1,1} + \left(q^{3} t + q^{2} t^{2} + q t^{3} + q^{2

In [18]:
A = wH([3,3],2)
cA=cleans(A)
latex(cA)

q^{3} t^{2} s_{} \otimes s_{1,1,1} + \left(q^{2} t + q t^{2}\right) s_{} \otimes s_{2,1} + t s_{} \otimes s_{3} + \left(q^{3} t + q^{2} t^{2} + q t\right) s_{1} \otimes s_{1,1} + \left(q^{2} + q t + t^{2}\right) s_{1} \otimes s_{2} + \left(q^{3} + q^{2} t + q t^{2}\right) s_{1,1} \otimes s_{1} + q^{3} t s_{1,1,1} \otimes s_{} + \left(q^{2} t + q + t\right) s_{2} \otimes s_{1} + \left(q^{2} + q t\right) s_{2,1} \otimes s_{} + s_{3} \otimes s_{}

In [19]:
A = wH([3,2],2)
cA=cleans(A)
latex(cA)

\frac{t}{q^{3}} s_{} \otimes s_{1,1} + \frac{1}{q^{2}} s_{} \otimes s_{2} + \left(\frac{q + t}{q^{2}}\right) s_{1} \otimes s_{1} + \frac{t}{q} s_{1,1} \otimes s_{} + s_{2} \otimes s_{}

In [22]:
A = wH([3,1],2)
cA=cleans(A)
latex(cA)

q t s_{} \otimes s_{1,1} + \frac{t}{q} s_{} \otimes s_{2} + \left(q + t\right) s_{1} \otimes s_{1} + q^{2} s_{1,1} \otimes s_{} + s_{2} \otimes s_{}

In [25]:
A = wH([2,2],2)
cA=cleans(A)
latex(cA)

q t s_{} \otimes s_{1,1} + s_{} \otimes s_{2} + \left(q + t\right) s_{1} \otimes s_{1} + q t s_{1,1} \otimes s_{} + s_{2} \otimes s_{}

In [26]:
A = wH([2,1,1],2)
cA=cleans(A)
latex(cA)

q t s_{} \otimes s_{1,1} + \frac{q}{t} s_{} \otimes s_{2} + \left(q + t\right) s_{1} \otimes s_{1} + t^{2} s_{1,1} \otimes s_{} + s_{2} \otimes s_{}

In [27]:
A = wH([3,1,1],2)
cA=cleans(A)
latex(cA)

\frac{1}{q t} s_{} \otimes s_{1,1} + \frac{1}{q t} s_{} \otimes s_{2} + \left(\frac{q + t}{q t}\right) s_{1} \otimes s_{1} + s_{1,1} \otimes s_{} + s_{2} \otimes s_{}

In [28]:
A = wH([4],2)
cA=cleans(A)
latex(cA)

q^{4} s_{} \otimes s_{1,1} + q^{2} s_{} \otimes s_{2} + \left(q^{3} + q\right) s_{1} \otimes s_{1} + q^{2} s_{1,1} \otimes s_{} + s_{2} \otimes s_{}

In [29]:
A = wH([3],2)
cA=cleans(A)
latex(cA)

\frac{1}{q} s_{} \otimes s_{1} + s_{1} \otimes s_{}

In [31]:
A = wH([2],2)
cA=cleans(A)
latex(cA)

q s_{} \otimes s_{1} + s_{1} \otimes s_{}

In [32]:
A = wH([2,2,2],2)
cA=cleans(A)
latex(cA)

q^{2} t^{3} s_{} \otimes s_{1,1,1} + \left(q^{2} t + q t^{2}\right) s_{} \otimes s_{2,1} + q s_{} \otimes s_{3} + \left(q^{2} t^{2} + q t^{3} + q t\right) s_{1} \otimes s_{1,1} + \left(q^{2} + q t + t^{2}\right) s_{1} \otimes s_{2} + \left(q^{2} t + q t^{2} + t^{3}\right) s_{1,1} \otimes s_{1} + q t^{3} s_{1,1,1} \otimes s_{} + \left(q t^{2} + q + t\right) s_{2} \otimes s_{1} + \left(q t + t^{2}\right) s_{2,1} \otimes s_{} + s_{3} \otimes s_{}

In [33]:
A = wH([2,2,1,1],2)
cA=cleans(A)
latex(cA)

q t^{4} s_{} \otimes s_{1,1,1} + \left(q t^{2} + t^{3}\right) s_{} \otimes s_{2,1} + t s_{} \otimes s_{3} + \left(q t^{3} + t^{4} + q t\right) s_{1} \otimes s_{1,1} + \left(q t + 2 t^{2}\right) s_{1} \otimes s_{2} + \left(2 q t^{2} + t^{3}\right) s_{1,1} \otimes s_{1} + q t^{3} s_{1,1,1} \otimes s_{} + \left(t^{3} + q + t\right) s_{2} \otimes s_{1} + \left(q t + t^{2}\right) s_{2,1} \otimes s_{} + s_{3} \otimes s_{}

In [34]:
A = wH([2,1,1,1,1],2)
cA=cleans(A)
latex(cA)

q t^{4} s_{} \otimes s_{1,1,1} + \left(q t^{2} + q\right) s_{} \otimes s_{2,1} + \frac{q}{t^{2}} s_{} \otimes s_{3} + \left(q t^{3} + t^{4} + q t\right) s_{1} \otimes s_{1,1} + \left(\frac{q t^{2} + t^{3} + q}{t}\right) s_{1} \otimes s_{2} + \left(t^{5} + q t^{2} + t^{3}\right) s_{1,1} \otimes s_{1} + t^{6} s_{1,1,1} \otimes s_{} + \left(t^{3} + q + t\right) s_{2} \otimes s_{1} + \left(t^{4} + t^{2}\right) s_{2,1} \otimes s_{} + s_{3} \otimes s_{}

In [35]:
A = wH([3,3],2)
cA=cleans(A)
latex(cA)

q^{3} t^{2} s_{} \otimes s_{1,1,1} + \left(q^{2} t + q t^{2}\right) s_{} \otimes s_{2,1} + t s_{} \otimes s_{3} + \left(q^{3} t + q^{2} t^{2} + q t\right) s_{1} \otimes s_{1,1} + \left(q^{2} + q t + t^{2}\right) s_{1} \otimes s_{2} + \left(q^{3} + q^{2} t + q t^{2}\right) s_{1,1} \otimes s_{1} + q^{3} t s_{1,1,1} \otimes s_{} + \left(q^{2} t + q + t\right) s_{2} \otimes s_{1} + \left(q^{2} + q t\right) s_{2,1} \otimes s_{} + s_{3} \otimes s_{}

In [37]:
A = wH([3,1,1,1],2)
cA=cleans(A)
latex(cA)

q t^{4} s_{} \otimes s_{1,1,1} + \left(\frac{q^{2} t^{2} + t^{4}}{q}\right) s_{} \otimes s_{2,1} + \frac{t^{2}}{q} s_{} \otimes s_{3} + \left(q t^{3} + t^{4} + q t\right) s_{1} \otimes s_{1,1} + \left(\frac{q^{2} t + q t^{2} + t^{3}}{q}\right) s_{1} \otimes s_{2} + \left(q^{2} t + q t^{2} + t^{3}\right) s_{1,1} \otimes s_{1} + q^{2} t^{2} s_{1,1,1} \otimes s_{} + \left(t^{3} + q + t\right) s_{2} \otimes s_{1} + \left(q^{2} + t^{2}\right) s_{2,1} \otimes s_{} + s_{3} \otimes s_{}

In [38]:
A = wH([4,2],2)
cA=cleans(A)
latex(cA)

q^{4} t s_{} \otimes s_{1,1,1} + \left(q^{3} + q^{2} t\right) s_{} \otimes s_{2,1} + q s_{} \otimes s_{3} + \left(q^{4} + q^{3} t + q t\right) s_{1} \otimes s_{1,1} + \left(2 q^{2} + q t\right) s_{1} \otimes s_{2} + \left(q^{3} + 2 q^{2} t\right) s_{1,1} \otimes s_{1} + q^{3} t s_{1,1,1} \otimes s_{} + \left(q^{3} + q + t\right) s_{2} \otimes s_{1} + \left(q^{2} + q t\right) s_{2,1} \otimes s_{} + s_{3} \otimes s_{}

In [40]:
A = wH([4,1,1],2)
cA=cleans(A)
latex(cA)

q^{4} t s_{} \otimes s_{1,1,1} + \left(\frac{q^{4} + q^{2} t^{2}}{t}\right) s_{} \otimes s_{2,1} + \frac{q^{2}}{t} s_{} \otimes s_{3} + \left(q^{4} + q^{3} t + q t\right) s_{1} \otimes s_{1,1} + \left(\frac{q^{3} + q^{2} t + q t^{2}}{t}\right) s_{1} \otimes s_{2} + \left(q^{3} + q^{2} t + q t^{2}\right) s_{1,1} \otimes s_{1} + q^{2} t^{2} s_{1,1,1} \otimes s_{} + \left(q^{3} + q + t\right) s_{2} \otimes s_{1} + \left(q^{2} + t^{2}\right) s_{2,1} \otimes s_{} + s_{3} \otimes s_{}

In [ ]:
r=3 Tries

In [41]:
A = wH([3,2,1],3)
cA=cleans(A)
latex(cA)

q t^{2} s_{} \otimes s_{} \otimes s_{1,1} + t s_{} \otimes s_{} \otimes s_{2} + 2 q t s_{} \otimes s_{1} \otimes s_{1} + q^{2} t s_{} \otimes s_{1,1} \otimes s_{} + q s_{} \otimes s_{2} \otimes s_{} + \left(t^{2} + q\right) s_{1} \otimes s_{} \otimes s_{1} + \left(q^{2} + t\right) s_{1} \otimes s_{1} \otimes s_{} + q t s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [42]:
A = wH([3,2,2],3)
cA=cleans(A)
latex(cA)

\frac{t}{q} s_{} \otimes s_{} \otimes s_{1} + t s_{} \otimes s_{1} \otimes s_{} + s_{1} \otimes s_{} \otimes s_{}

In [43]:
A = wH([3,3,2],3)
cA=cleans(A)
latex(cA)

q s_{} \otimes s_{} \otimes s_{1} + t s_{} \otimes s_{1} \otimes s_{} + s_{1} \otimes s_{} \otimes s_{}

In [18]:
A = wH([3,3],2)
cA=cleans(A)
latex(cA)

q^{3} t^{2} s_{} \otimes s_{1,1,1} + \left(q^{2} t + q t^{2}\right) s_{} \otimes s_{2,1} + t s_{} \otimes s_{3} + \left(q^{3} t + q^{2} t^{2} + q t\right) s_{1} \otimes s_{1,1} + \left(q^{2} + q t + t^{2}\right) s_{1} \otimes s_{2} + \left(q^{3} + q^{2} t + q t^{2}\right) s_{1,1} \otimes s_{1} + q^{3} t s_{1,1,1} \otimes s_{} + \left(q^{2} t + q + t\right) s_{2} \otimes s_{1} + \left(q^{2} + q t\right) s_{2,1} \otimes s_{} + s_{3} \otimes s_{}

In [44]:
A = wH([3,2],3)
cA=cleans(A)
latex(cA)

\frac{1}{t} s_{} \otimes s_{} \otimes s_{1} + \frac{q}{t} s_{} \otimes s_{1} \otimes s_{} + s_{1} \otimes s_{} \otimes s_{}

In [45]:
A = wH([3,1],3)
cA=cleans(A)
latex(cA)

s_{} \otimes s_{} \otimes s_{}

In [63]:
A = wH([2,2],3)
cA=cleans(A)
latex(cA)

\frac{1}{t} s_{} \otimes s_{} \otimes s_{1} + \frac{1}{q} s_{} \otimes s_{1} \otimes s_{} + s_{1} \otimes s_{} \otimes s_{}

In [64]:
A = wH([2,1,1],3)
cA=cleans(A)
latex(cA)

s_{} \otimes s_{} \otimes s_{}

In [68]:
A = wH([3,2],3)
cA=cleans(A)
latex(cA)

\frac{1}{t} s_{} \otimes s_{} \otimes s_{1} + \frac{q}{t} s_{} \otimes s_{1} \otimes s_{} + s_{1} \otimes s_{} \otimes s_{}

In [69]:
A = wH([4],3)
cA=cleans(A)
latex(cA)

\frac{1}{q^{2}} s_{} \otimes s_{} \otimes s_{1} + \frac{1}{q} s_{} \otimes s_{1} \otimes s_{} + s_{1} \otimes s_{} \otimes s_{}

In [59]:
A = wH([3],3)
cA=cleans(A)
latex(cA)

q s_{} \otimes s_{} \otimes s_{1} + q^{2} s_{} \otimes s_{1} \otimes s_{} + s_{1} \otimes s_{} \otimes s_{}

In [61]:
A = wH([2,1],3)
cA=cleans(A)
latex(cA)

q s_{} \otimes s_{} \otimes s_{1} + t s_{} \otimes s_{1} \otimes s_{} + s_{1} \otimes s_{} \otimes s_{}

In [52]:
A = wH([2,2,2],3)
cA=cleans(A)
latex(cA)

q t^{2} s_{} \otimes s_{} \otimes s_{1,1} + t s_{} \otimes s_{} \otimes s_{2} + \left(t^{3} + q t\right) s_{} \otimes s_{1} \otimes s_{1} + q t^{3} s_{} \otimes s_{1,1} \otimes s_{} + t^{2} s_{} \otimes s_{2} \otimes s_{} + \left(t^{2} + q\right) s_{1} \otimes s_{} \otimes s_{1} + \left(q t^{2} + t\right) s_{1} \otimes s_{1} \otimes s_{} + q t s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [53]:
A = wH([2,2,1,1],3)
cA=cleans(A)
latex(cA)

s_{} \otimes s_{} \otimes s_{}

In [54]:
A = wH([2,1,1,1,1],3)
cA=cleans(A)
latex(cA)

q t^{2} s_{} \otimes s_{} \otimes s_{1,1} + \frac{q}{t} s_{} \otimes s_{} \otimes s_{2} + \left(t^{3} + q t\right) s_{} \otimes s_{1} \otimes s_{1} + t^{5} s_{} \otimes s_{1,1} \otimes s_{} + t^{2} s_{} \otimes s_{2} \otimes s_{} + \left(t^{2} + q\right) s_{1} \otimes s_{} \otimes s_{1} + \left(t^{4} + t\right) s_{1} \otimes s_{1} \otimes s_{} + t^{3} s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [55]:
A = wH([3,3],3)
cA=cleans(A)
latex(cA)

q^{3} t s_{} \otimes s_{} \otimes s_{1,1} + q^{2} s_{} \otimes s_{} \otimes s_{2} + \left(q^{3} + q t\right) s_{} \otimes s_{1} \otimes s_{1} + q^{2} t s_{} \otimes s_{1,1} \otimes s_{} + q s_{} \otimes s_{2} \otimes s_{} + \left(q^{2} t + q\right) s_{1} \otimes s_{} \otimes s_{1} + \left(q^{2} + t\right) s_{1} \otimes s_{1} \otimes s_{} + q t s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [56]:
A = wH([3,1,1,1],3)
cA=cleans(A)
latex(cA)

q t^{2} s_{} \otimes s_{} \otimes s_{1,1} + \frac{q}{t} s_{} \otimes s_{} \otimes s_{2} + \left(\frac{q t^{2} + q^{2}}{t}\right) s_{} \otimes s_{1} \otimes s_{1} + q^{2} t s_{} \otimes s_{1,1} \otimes s_{} + \frac{q^{2}}{t^{2}} s_{} \otimes s_{2} \otimes s_{} + \left(t^{2} + q\right) s_{1} \otimes s_{} \otimes s_{1} + \left(q^{2} + t\right) s_{1} \otimes s_{1} \otimes s_{} + t^{3} s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [71]:
A = wH([4,1],3)
cA=cleans(A)
latex(cA)

\frac{1}{q^{2}} s_{} \otimes s_{} \otimes s_{1} + \frac{1}{q} s_{} \otimes s_{1} \otimes s_{} + s_{1} \otimes s_{} \otimes s_{}

In [73]:
A = wH([4,1,1],3)
cA=cleans(A)
latex(cA)

q t^{2} s_{} \otimes s_{} \otimes s_{1,1} + \frac{t^{2}}{q^{2}} s_{} \otimes s_{} \otimes s_{2} + \left(\frac{q^{2} t + t^{2}}{q}\right) s_{} \otimes s_{1} \otimes s_{1} + q^{2} t s_{} \otimes s_{1,1} \otimes s_{} + \frac{t}{q} s_{} \otimes s_{2} \otimes s_{} + \left(t^{2} + q\right) s_{1} \otimes s_{} \otimes s_{1} + \left(q^{2} + t\right) s_{1} \otimes s_{1} \otimes s_{} + q^{3} s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [75]:
A = wH([4,4],3)
cA=cleans(A)
latex(cA)

t s_{} \otimes s_{} \otimes s_{1,1} + \frac{1}{q} s_{} \otimes s_{} \otimes s_{2} + \left(q t + 1\right) s_{} \otimes s_{1} \otimes s_{1} + q^{2} t s_{} \otimes s_{1,1} \otimes s_{} + q s_{} \otimes s_{2} \otimes s_{} + \left(\frac{q^{2} + t}{q}\right) s_{1} \otimes s_{} \otimes s_{1} + \left(q^{2} + t\right) s_{1} \otimes s_{1} \otimes s_{} + q t s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [76]:
A = wH([2,2,2],3)
cA=cleans(A)
latex(cA)

q t^{2} s_{} \otimes s_{} \otimes s_{1,1} + t s_{} \otimes s_{} \otimes s_{2} + \left(t^{3} + q t\right) s_{} \otimes s_{1} \otimes s_{1} + q t^{3} s_{} \otimes s_{1,1} \otimes s_{} + t^{2} s_{} \otimes s_{2} \otimes s_{} + \left(t^{2} + q\right) s_{1} \otimes s_{} \otimes s_{1} + \left(q t^{2} + t\right) s_{1} \otimes s_{1} \otimes s_{} + q t s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [77]:
A = wH([3,3],3)
cA=cleans(A)
latex(cA)

q^{3} t s_{} \otimes s_{} \otimes s_{1,1} + q^{2} s_{} \otimes s_{} \otimes s_{2} + \left(q^{3} + q t\right) s_{} \otimes s_{1} \otimes s_{1} + q^{2} t s_{} \otimes s_{1,1} \otimes s_{} + q s_{} \otimes s_{2} \otimes s_{} + \left(q^{2} t + q\right) s_{1} \otimes s_{} \otimes s_{1} + \left(q^{2} + t\right) s_{1} \otimes s_{1} \otimes s_{} + q t s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [78]:
A = wH([5],3)
cA=cleans(A)
latex(cA)

q s_{} \otimes s_{} \otimes s_{1} + \frac{1}{q} s_{} \otimes s_{1} \otimes s_{} + s_{1} \otimes s_{} \otimes s_{}

In [80]:
A = wH([5,1],3)
cA=cleans(A)
latex(cA)

q^{5} s_{} \otimes s_{} \otimes s_{1,1} + q^{2} s_{} \otimes s_{} \otimes s_{2} + \left(q^{3} + q t\right) s_{} \otimes s_{1} \otimes s_{1} + q^{2} t s_{} \otimes s_{1,1} \otimes s_{} + \frac{t}{q} s_{} \otimes s_{2} \otimes s_{} + \left(q^{4} + q\right) s_{1} \otimes s_{} \otimes s_{1} + \left(q^{2} + t\right) s_{1} \otimes s_{1} \otimes s_{} + q^{3} s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [1]:
1+1

2

In [4]:
A = wH([3,2,1],3)

In [5]:
cA = cleans(A)

In [6]:
latex(A)

q t^{2} s_{} \otimes s_{} \otimes s_{1,1} + t s_{} \otimes s_{} \otimes s_{2} + 2 q t s_{} \otimes s_{1} \otimes s_{1} + q^{2} t s_{} \otimes s_{1,1} \otimes s_{} + q s_{} \otimes s_{2} \otimes s_{} + \left(t^{2} + q\right) s_{1} \otimes s_{} \otimes s_{1} + \left(q^{2} + t\right) s_{1} \otimes s_{1} \otimes s_{} + q t s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [7]:
latex(cA)

q t^{2} s_{} \otimes s_{} \otimes s_{1,1} + t s_{} \otimes s_{} \otimes s_{2} + 2 q t s_{} \otimes s_{1} \otimes s_{1} + q^{2} t s_{} \otimes s_{1,1} \otimes s_{} + q s_{} \otimes s_{2} \otimes s_{} + \left(t^{2} + q\right) s_{1} \otimes s_{} \otimes s_{1} + \left(q^{2} + t\right) s_{1} \otimes s_{1} \otimes s_{} + q t s_{1,1} \otimes s_{} \otimes s_{} + s_{2} \otimes s_{} \otimes s_{}

In [29]:
B = wH([3,3,3],3)

In [30]:
latex(B)

q^{3} t^{3} s_{} \otimes s_{} \otimes s_{1,1,1} + \left(q^{2} t^{2} + q t\right) s_{} \otimes s_{} \otimes s_{2,1} + s_{} \otimes s_{} \otimes s_{3} + \left(q^{3} t^{2} + q t^{3} + q^{2} t\right) s_{} \otimes s_{1} \otimes s_{1,1} + \left(q^{2} t + t^{2} + q\right) s_{} \otimes s_{1} \otimes s_{2} + \left(q^{2} t^{3} + q^{3} t + q t^{2}\right) s_{} \otimes s_{1,1} \otimes s_{1} + q^{3} t^{3} s_{} \otimes s_{1,1,1} \otimes s_{} + \left(q t^{2} + q^{2} + t\right) s_{} \otimes s_{2} \otimes s_{1} + \left(q^{2} t^{2} + q t\right) s_{} \otimes s_{2,1} \otimes s_{} + s_{} \otimes s_{3} \otimes s_{} + \left(q^{2} t^{3} + q^{3} t + q t^{2}\right) s_{1} \otimes s_{} \otimes s_{1,1} + \left(q t^{2} + q^{2} + t\right) s_{1} \otimes s_{} \otimes s_{2} + \left(2 q^{2} t^{2} + q^{3} + t^{3} + 2 q t\right) s_{1} \otimes s_{1} \otimes s_{1} + \left(q^{3} t^{2} + q t^{3} + q^{2} t\right) s_{1} \otimes s_{1,1} \otimes s_{} + \left(q^{2} t + t^{2} + q\right) s_{1} \otimes s_{2} \otimes s_{} + \left(q^{3}

In [ ]:
cleans(B)

In [53]:
C = wH([5,4,2,1],4)

In [54]:
latex(C)

q^{3} t^{4} s_{} \otimes s_{} \otimes s_{} \otimes s_{1,1,1} + \left(\frac{q^{3} t^{3} + t^{4}}{q}\right) s_{} \otimes s_{} \otimes s_{} \otimes s_{2,1} - \left(\frac{t^{3}}{-q^{2}}\right) s_{} \otimes s_{} \otimes s_{} \otimes s_{3} + \left(2 q^{3} t^{3} + t^{4}\right) s_{} \otimes s_{} \otimes s_{1} \otimes s_{1,1} - \left(\frac{q^{3} t^{2} + q t^{4} + t^{3}}{-q}\right) s_{} \otimes s_{} \otimes s_{1} \otimes s_{2} + \left(q^{4} t^{3} + q^{3} t^{2} + q t^{4}\right) s_{} \otimes s_{} \otimes s_{1,1} \otimes s_{1} + q^{5} t^{3} s_{} \otimes s_{} \otimes s_{1,1,1} \otimes s_{} + \left(q^{3} t^{2} + 2 t^{3}\right) s_{} \otimes s_{} \otimes s_{2} \otimes s_{1} + \left(q^{4} t^{2} + q t^{3}\right) s_{} \otimes s_{} \otimes s_{2,1} \otimes s_{} + t^{2} s_{} \otimes s_{} \otimes s_{3} \otimes s_{} + \left(q^{3} t^{2} + 2 q t^{4}\right) s_{} \otimes s_{1} \otimes s_{} \otimes s_{1,1} - \left(\frac{q^{4} t + q^{2} t^{3} + t^{5}}{-q^{2}}\right) s_{} \otimes s_{1} \otimes s_{} \otimes s_{2} - \l

In [42]:
cleans(C)

q^3*t^4*s[] # s[] # s[] # s[1, 1, 1] + (q^2*t^3+q^3)*s[] # s[] # s[] # s[2, 1] + q^2/t*s[] # s[] # s[] # s[3] + (q^3*t^3+q*t^5+q^4)*s[] # s[] # s[1] # s[1, 1] + ((2*q^2*t^3+q^3)/t)*s[] # s[] # s[1] # s[2] + (q^2*t^5+2*q^3*t^2)*s[] # s[] # s[1, 1] # s[1] + q^3*t^5*s[] # s[] # s[1, 1, 1] # s[] + ((q*t^5+q^4+q^2*t^2)/t)*s[] # s[] # s[2] # s[1] + (q^2*t^4+q^3*t)*s[] # s[] # s[2, 1] # s[] + q^2*s[] # s[] # s[3] # s[] + (q^5+q^3*t^2+q*t^4)*s[] # s[1] # s[] # s[1, 1] + ((q^4+2*q^2*t^2)/t)*s[] # s[1] # s[] # s[2] + ((q^2*t^5+q^5+3*q^3*t^2+q*t^4)/t)*s[] # s[1] # s[1] # s[1] + (q^3*t^4+q^4*t+q^2*t^3)*s[] # s[1] # s[1, 1] # s[] + (q^2*t^3+2*q^3)*s[] # s[1] # s[2] # s[] + (2*q^4*t+q^2*t^3)*s[] # s[1, 1] # s[] # s[1] + (2*q^3*t^3+q^4)*s[] # s[1, 1] # s[1] # s[] + q^4*t^3*s[] # s[1, 1, 1] # s[] # s[] + ((q^5+q^3*t^2+q*t^4)/t^2)*s[] # s[2] # s[] # s[1] + ((q^4*t+q^2*t^3+q^3)/t)*s[] # s[2] # s[1] # s[] + ((q^3*t^3+q^4)/t)*s[] # s[2, 1] # s[] # s[] + q^3/t^2*s[] # s[3] # s[] # s[] + (q^2*t^4+q^3*t+q*t^3)*s[1] # s[] # s[] # s[1, 1] + (q*t^3+2*q^2)*s[1] # s[] # s[] # s[2] + (2*q^2*t^3+t^5+2*q^3+q*t^2)*s[1] # s[] # s[1] # s[1] + (q^3*t^3+q*t^5+q^2*t^2)*s[1] # s[] # s[1, 1] # s[] + ((q^2*t^3+t^5+q^3)/t)*s[1] # s[] # s[2] # s[] + (2*q^4+2*q^2*t^2+t^4+q*t)*s[1] # s[1] # s[] # s[1] + ((2*q^3*t^3+q*t^5+q^4+q^2*t^2+t^4)/t)*s[1] # s[1] # s[1] # s[] + (q^4*t^2+q^3*t+q*t^3)*s[1] # s[1, 1] # s[] # s[] + ((q^3*t^3+q^4+t^4)/t^2)*s[1] # s[2] # s[] # s[] + (2*q*t^4+q^2*t)*s[1, 1] # s[] # s[] # s[1] + (t^6+q^3*t+q*t^3)*s[1, 1] # s[] # s[1] # s[] + (q^4*t+t^5+q*t^2)*s[1, 1] # s[1] # s[] # s[] + q*t^5*s[1, 1, 1] # s[] # s[] # s[] + (q^2*t+t^3+q)*s[2] # s[] # s[] # s[1] + (q*t^3+q^2+t^2)*s[2] # s[] # s[1] # s[] + (q^3+q*t^2+t)*s[2] # s[1] # s[] # s[] + (t^4+q*t)*s[2, 1] # s[] # s[] # s[] + s[3] # s[] # s[] # s[]

In [48]:
latex(top(C))

-\left(\frac{\frac{1}{6} q^{3} t^{5} + \frac{1}{3} q^{2} t^{4} + \frac{1}{3} q^{3} t + \frac{1}{6} q^{2}}{-t}\right) p_{} \otimes p_{} \otimes p_{} \otimes p_{1,1,1} + \left(\frac{\frac{1}{2} q^{3} t^{5} - \frac{1}{2} q^{2}}{-t}\right) p_{} \otimes p_{} \otimes p_{} \otimes p_{2,1} - \left(\frac{\frac{1}{3} q^{3} t^{5} - \frac{1}{3} q^{2} t^{4} - \frac{1}{3} q^{3} t + \frac{1}{3} q^{2}}{-t}\right) p_{} \otimes p_{} \otimes p_{} \otimes p_{3} + \left(\frac{\frac{1}{2} q^{3} t^{4} + \frac{1}{2} q t^{6} + \frac{1}{2} q^{4} t + q^{2} t^{3} + \frac{1}{2} q^{3}}{t}\right) p_{} \otimes p_{} \otimes p_{1} \otimes p_{1,1} - \left(\frac{\frac{1}{2} q^{3} t^{4} + \frac{1}{2} q t^{6} + \frac{1}{2} q^{4} t - q^{2} t^{3} - \frac{1}{2} q^{3}}{t}\right) p_{} \otimes p_{} \otimes p_{1} \otimes p_{2} - \left(\frac{\frac{1}{2} q^{2} t^{6} + q^{3} t^{3} + \frac{1}{2} q t^{5} + \frac{1}{2} q^{4} + \frac{1}{2} q^{2} t^{2}}{-t}\right) p_{} \otimes p_{} \otimes p_{1,1} \otimes p_{1} + \left(\frac{1}{6} q^{3} 

In [201]:
simpletensor(e[3]*e[2],2,5)

p[] # p[] # e[3, 2] # p[] # p[]

In [ ]:
def modifiedpk(k,a,r):
    return sum([p[k] for i in range(r)])


In [198]:
tensor([p[0],p[0]])

p[] # p[]

In [178]:
hscalar(wH([6,2],3),wH([3,2,2,1],3))

(-q^7*t^2 - q^5*t^2 - q^4*t^3 - q^2*t^5 - q^6 - q^5*t - q^2*t^4 - q^5 - 2*q^3*t^2 - q^2*t^3 - q^3*t - q^2*t^2 - t^4 - q^3 - q*t^2 - t^3 - q)/(-t^3)

In [179]:
factor(starscalar(wH([6,2],3),wH([3,2,2,1],3)))

(1/2) * t^-3 * (t - 1)^-1 * q^-1 * (q - 1)^-1 * (t^2 + t + 1)^-1 * (q^2 + q + 1)^-1 * (q^2*t^2 + q*t + 1) * (q^9 + 2*q^6*t^2 + 2*q^5*t^3 + 2*q^7 + 2*q^6*t + q^5*t^2 + q^4*t^3 + 2*q^3*t^4 + 2*q^4*t^2 + 2*q^3*t^3 + q^5 + 2*q^4*t + 2*q^3*t^2 + 2*q*t^4 + t^5 + 2*q^4 + 2*q^2*t^2 + q*t^2)

In [ ]:
F = wH([5,4,3],3)
cleans(F)

In [258]:
cleans(dagger(F)*q^8*t^3)

q^7*t^3*s[] # s[] # s[1, 1, 1, 1] + (q^6*t^4+q^5*t^5+q^4*t^3)*s[] # s[] # s[2, 1, 1] + (q^5*t^5+q^3*t^4)*s[] # s[] # s[2, 2] + (q^4*t^6+q^3*t^4+q^2*t^5)*s[] # s[] # s[3, 1] + q*t^6*s[] # s[] # s[4] + (q^7*t^5+q^6*t^3+2*q^5*t^4)*s[] # s[1] # s[1, 1, 1] + (2*q^6*t^6+q^5*t^4+3*q^4*t^5+q^3*t^3+q^2*t^4)*s[] # s[1] # s[2, 1] + (q^5*t^7+q^3*t^6+q^2*t^4+q*t^5)*s[] # s[1] # s[3] + (q^8*t^6+2*q^6*t^5+q^5*t^3+2*q^4*t^4)*s[] # s[1, 1] # s[1, 1] + (q^7*t^7+q^6*t^5+q^5*t^6+q^4*t^4+q^3*t^5+q^2*t^3)*s[] # s[1, 1] # s[2] + (q^8*t^5+q^7*t^6+q^6*t^4+q^4*t^3)*s[] # s[1, 1, 1] # s[1] + q^8*t^4*s[] # s[1, 1, 1, 1] # s[] + (q^6*t^5+2*q^5*t^6+2*q^4*t^4+q^3*t^5)*s[] # s[2] # s[1, 1] + (2*q^5*t^6+q^4*t^7+2*q^3*t^5+q*t^4)*s[] # s[2] # s[2] + (q^7*t^6+q^6*t^7+3*q^5*t^5+q^4*t^6+2*q^3*t^4)*s[] # s[2, 1] # s[1] + (q^7*t^5+q^6*t^6+q^5*t^4)*s[] # s[2, 1, 1] # s[] + (q^6*t^6+q^4*t^5)*s[] # s[2, 2] # s[] + (2*q^4*t^6+q^3*t^7+q^2*t^5)*s[] # s[3] # s[1] + (q^5*t^7+q^4*t^5+q^3*t^6)*s[] # s[3, 1] # s[] + q^2*t^7*s[] # s[4] # s[] + (q^7*t^4+q^6*t^5+2*q^5*t^3)*s[1] # s[] # s[1, 1, 1] + (2*q^6*t^5+q^5*t^6+3*q^4*t^4+q^3*t^5+q^2*t^3)*s[1] # s[] # s[2, 1] + (q^5*t^6+q^3*t^5+q^2*t^6+q*t^4)*s[1] # s[] # s[3] + (2*q^7*t^6+2*q^6*t^4+4*q^5*t^5+2*q^4*t^3+2*q^3*t^4)*s[1] # s[1] # s[1, 1] + (q^7*t^6+q^6*t^7+3*q^5*t^5+2*q^4*t^6+3*q^3*t^4+q^2*t^5+q*t^3)*s[1] # s[1] # s[2] + (q^8*t^7+2*q^7*t^5+2*q^6*t^6+3*q^5*t^4+2*q^4*t^5+2*q^3*t^3)*s[1] # s[1, 1] # s[1] + (q^8*t^6+q^7*t^4+q^6*t^5+q^5*t^3)*s[1] # s[1, 1, 1] # s[] + (3*q^6*t^6+q^5*t^7+q^5*t^4+4*q^4*t^5+q^3*t^6+2*q^2*t^4)*s[1] # s[2] # s[1] + (q^7*t^7+2*q^6*t^5+2*q^5*t^6+2*q^4*t^4+q^3*t^5)*s[1] # s[2, 1] # s[] + (q^5*t^6+q^4*t^7+q^3*t^5+q^2*t^6)*s[1] # s[3] # s[] + (2*q^7*t^5+q^6*t^3+2*q^5*t^4+q^3*t^3)*s[1, 1] # s[] # s[1, 1] + (q^7*t^5+q^6*t^6+q^5*t^4+q^4*t^5+q^3*t^3+q^2*t^4)*s[1, 1] # s[] # s[2] + (q^8*t^6+q^7*t^4+4*q^6*t^5+q^5*t^6+q^5*t^3+3*q^4*t^4+q^2*t^3)*s[1, 1] # s[1] # s[1] + (q^8*t^5+q^7*t^6+2*q^6*t^4+q^5*t^5+q^4*t^3)*s[1, 1] # s[1, 1] # s[] + (q^7*t^6+q^6*t^4+2*q^5*t^5+q^4*t^6+q^3*t^4)*s[1, 1] # s[2] # s[] + (q^8*t^5+q^7*t^3+q^6*t^4+q^4*t^3)*s[1, 1, 1] # s[] # s[1] + (q^8*t^4+q^7*t^5+q^6*t^3+q^5*t^4)*s[1, 1, 1] # s[1] # s[] + q^8*t^3*s[1, 1, 1, 1] # s[] # s[] + (q^6*t^6+2*q^5*t^4+2*q^4*t^5+q^3*t^3)*s[2] # s[] # s[1, 1] + (q^6*t^6+2*q^4*t^5+q^3*t^6+2*q^2*t^4)*s[2] # s[] # s[2] + (q^7*t^7+q^6*t^5+3*q^5*t^6+3*q^4*t^4+3*q^3*t^5+q^2*t^3)*s[2] # s[1] # s[1] + (q^7*t^6+q^6*t^7+2*q^5*t^5+q^4*t^3+q^3*t^4)*s[2] # s[1, 1] # s[] + (q^6*t^7+q^5*t^5+2*q^4*t^6+q^3*t^4+q^2*t^5)*s[2] # s[2] # s[] + (q^7*t^6+q^6*t^4+3*q^5*t^5+q^4*t^3+2*q^3*t^4)*s[2, 1] # s[] # s[1] + (q^7*t^5+2*q^6*t^6+2*q^5*t^4+2*q^4*t^5+q^3*t^3)*s[2, 1] # s[1] # s[] + (q^7*t^4+q^6*t^5+q^5*t^3)*s[2, 1, 1] # s[] # s[] + (q^6*t^5+q^4*t^4)*s[2, 2] # s[] # s[] + (2*q^4*t^6+q^3*t^4+q^2*t^5)*s[3] # s[] # s[1] + (q^5*t^7+q^4*t^5+q^3*t^6+q^2*t^4)*s[3] # s[1] # s[] + (q^5*t^6+q^4*t^4+q^3*t^5)*s[3, 1] # s[] # s[] + q^2*t^6*s[4] # s[] # s[]

In [ ]:
def modifiedpi(i):
    for a in range(r)

In [29]:
list(tos(wH([4,2,1],3)))

[(([1, 1], [], []), (-t^2)/(-q)),
 (([2], [], []), 1),
 (([1], [1], []), (-t^2 - q)/(-q^2)),
 (([1], [], [1]), (-q^2*t - t^2)/(-q^3)),
 (([], [1, 1], []), t^2/q^3),
 (([], [2], []), (-1)/(-q^2)),
 (([], [1], [1]), (-q^2*t - t^2)/(-q^4)),
 (([], [], [1, 1]), t^3/q^4),
 (([], [], [2]), (-t)/(-q^3))]

In [2]:
wH([4,2,1],3)

t^3/q^4*s[] # s[] # s[1, 1] + ((-t)/(-q^3))*s[] # s[] # s[2] + ((-q^2*t-t^2)/(-q^4))*s[] # s[1] # s[1] + t^2/q^3*s[] # s[1, 1] # s[] - (1/(-q^2))*s[] # s[2] # s[] + ((-q^2*t-t^2)/(-q^3))*s[1] # s[] # s[1] + ((-t^2-q)/(-q^2))*s[1] # s[1] # s[] + ((-t^2)/(-q))*s[1, 1] # s[] # s[] + s[2] # s[] # s[]

In [31]:
k_predecessors(Partition([4,4,4]),1,3)

[[4, 4, 1], [4, 3, 2], [3, 3, 3]]

In [47]:
A = wH([4,4,1],3)
TA= Total_Tmu(Partition([4,4,1]))
TA

q^12*t^6

In [48]:
B = wH([3,3,3],3)
TB =Total_Tmu(Partition([3,3,3]))
TB

q^9*t^9

In [49]:
tos((TB*A-TA*B)/(TB-TA),3)

q^3*t^3*s[] # s[] # s[1, 1, 1] + ((-q^5*t^2+q^2*t^5+t^6-q^4*t)/(-q^3+t^3))*s[] # s[] # s[2, 1] + ((t^5-q^4)/(-q^4+q*t^3))*s[] # s[] # s[3] + ((-q^6*t^2+2*q^3*t^5-q^4*t^3+q*t^6-q^5*t)/(-q^3+t^3))*s[] # s[1] # s[1, 1] + ((q*t^6-q^5*t+q^2*t^4-q^3*t^2+t^5-q^4)/(-q^3+t^3))*s[] # s[1] # s[2] + ((q^4*t^5-q^5*t^3+q^2*t^6-q^6*t+q^3*t^4-q^4*t^2)/(-q^3+t^3))*s[] # s[1, 1] # s[1] + ((q^5*t^5-q^6*t^3)/(-q^3+t^3))*s[] # s[1, 1, 1] # s[] + ((q^3*t^4-q^4*t^2+2*q*t^5-q^5-q^3*t)/(-q^3+t^3))*s[] # s[2] # s[1] + ((q^4*t^4-q^5*t^2+q^2*t^5-q^4*t)/(-q^3+t^3))*s[] # s[2, 1] # s[] + ((q*t^4-q^3)/(-q^3+t^3))*s[] # s[3] # s[] + (q^2*t^3+q^3*t+q*t^2)*s[1] # s[] # s[1, 1] + ((-q^5*t^2+q^2*t^5-q^6+q^3*t^3+t^6-q^4*t)/(-q^4+q*t^3))*s[1] # s[] # s[2] + ((q^4*t^4-2*q^5*t^2+2*q^2*t^5-q^6+t^6-2*q^4*t+q*t^4)/(-q^3+t^3))*s[1] # s[1] # s[1] + ((q^5*t^4-q^6*t^2+q^3*t^5-q^4*t^3-q^5*t+q^2*t^4)/(-q^3+t^3))*s[1] # s[1, 1] # s[] + ((q^4*t^3-q^5*t+q^2*t^4-q^3*t^2+t^5-q^4)/(-q^3+t^3))*s[1] # s[2] # s[] + ((-q^6*t^2+q*t^6-q^5*t+q^2*

In [42]:
(((-2*q^3*t^4+q^4*t^2-q*t^5+q^2*t^3+q^3*t)/(-t^2+q))).factor()

t * q * (t^2 - q)^-1 * (2*q^2*t^3 - q^3*t + t^4 - q*t^2 - q^2)

In [ ]:
A=wH([7,6,6,4,3,2,2,1],4)
A

In [75]:
toHt(A,3)

((q^2*t-q^2-t+1)/(-q*t^2+t^3+q^2-q*t))*McdHt[] # McdHt[] # McdHt[1, 1, 1, 1] + ((-q^3*t^3-q^4*t+q^2*t^3+q^4+q^3*t-2*q^2*t^2+q*t^3+t^4+q^2*t-t^3-q*t)/(q^4-q^3*t-q^2*t^2+q*t^3))*McdHt[] # McdHt[] # McdHt[2, 1, 1] + ((q^4*t^5-q^4*t^4-q^3*t^5-q^5*t^2+q^3*t^4-q*t^6+2*q^5*t-2*q^4*t^2+2*q^3*t^3+2*q^2*t^4-2*q*t^5+2*t^6-q^4*t+q^2*t^3-t^5-q^2*t^2-q*t^3+q*t^2)/(q^5*t^2-2*q^4*t^3+q^3*t^4-q^6+2*q^5*t-q^4*t^2-q^3*t^3+2*q^2*t^4-q*t^5+q^4*t-2*q^3*t^2+q^2*t^3))*McdHt[] # McdHt[] # McdHt[2, 2] + ((q^3*t^4-q^3*t^3+q*t^5-q^4*t-q^3*t^2+2*q^2*t^3-q*t^4-t^5+q^3*t-q*t^3+q*t^2)/(-q^5+q^4*t+q^3*t^2-q^2*t^3))*McdHt[] # McdHt[] # McdHt[3, 1] + ((-q*t^4+t^4+q*t^2-t^2)/(-q^5+q^4*t+q^3*t-q^2*t^2))*McdHt[] # McdHt[] # McdHt[4] + ((-q^2*t^4-q^3*t^2+2*q^2*t^3-t^5+2*q^3*t-q^2*t^2-q*t^3+2*t^4-q^3+2*q*t^2-t^3-q*t)/(q*t^3-t^4-q^2*t+q*t^2))*McdHt[] # McdHt[1] # McdHt[1, 1, 1] + ((q^4*t^3+q^3*t^4+q^5*t-q^4*t^2-q^2*t^4+q*t^5-2*q^5-2*t^5+q^4-q^3*t-q*t^3+t^4+q^2*t+q*t^2)/(q^3*t^2-q^4-q*t^3+q^2*t))*McdHt[] # McdHt[1] # McdHt[2, 

In [52]:
tensor([m,m,m])(tensor([s[2,1],s[1],s[0]]))

2*m[1, 1, 1] # m[1] # m[] + m[2, 1] # m[1] # m[]

In [11]:
Partition([4,4,3,2]).quotient(3)

([1, 1], [], [2])

In [12]:
Partition([4,4,3,2]).core(3)

[1]

In [80]:
for la, coeff in Ht(s[3,2]):
    print(la)
    print(coeff)

[4, 1]
t^2/(q^6 - q^5*t - q^4*t + q^2*t^3 + q*t^3 - t^4)
[3, 2]
t/(-q^4 + 2*q^3*t - q^2*t^2 + q^2*t - 2*q*t^2 + t^3)
[3, 1, 1]
(-q^4*t - q*t^4 + q^3*t + q*t^3)/(-q^5*t^3 + 2*q^4*t^4 - q^3*t^5 + q^7 - 2*q^6*t + q^5*t^2 + q^2*t^5 - 2*q*t^6 + t^7 - q^4*t^2 + 2*q^3*t^3 - q^2*t^4)
[2, 2, 1]
q/(-q^2*t^2 + 2*q*t^3 - t^4 + q^3 - 2*q^2*t + q*t^2)
[2, 1, 1, 1]
(-q^2)/(q*t^5 - t^6 - q^3*t^2 + q*t^4 + q^4 - q^3*t)


In [81]:
Ht(s[3,2])

((-q^2)/(q*t^5-t^6-q^3*t^2+q*t^4+q^4-q^3*t))*McdHt[2, 1, 1, 1] + (q/(-q^2*t^2+2*q*t^3-t^4+q^3-2*q^2*t+q*t^2))*McdHt[2, 2, 1] + ((-q^4*t-q*t^4+q^3*t+q*t^3)/(-q^5*t^3+2*q^4*t^4-q^3*t^5+q^7-2*q^6*t+q^5*t^2+q^2*t^5-2*q*t^6+t^7-q^4*t^2+2*q^3*t^3-q^2*t^4))*McdHt[3, 1, 1] + (t/(-q^4+2*q^3*t-q^2*t^2+q^2*t-2*q*t^2+t^3))*McdHt[3, 2] + (t^2/(q^6-q^5*t-q^4*t+q^2*t^3+q*t^3-t^4))*McdHt[4, 1]

In [93]:
def Tmu(la):
    T = 1
    for i in range(len(la)):
        for j in range(la[i]):
            T= T * t^i * q^j
    return T

def nabla_op(f):
    QQ = 0
    PP = Ht(f)
    for la, coeff in PP:
        QQ = QQ + coeff * Ht(la)* Tmu(la)
    return(QQ)
        

In [107]:
nabla_op(e[2])


(t/(-q+t))*McdHt[1, 1] + ((-q)/(-q+t))*McdHt[2]

In [98]:
def qtCat(n):
    return nabla_op(e[n]).scalar(e[n])

In [106]:
qtCat(7).subs(q=1).subs(t=1)

429

In [74]:
tensor([Ht,Ht,Ht])(tensor([s[2,1],s[2],s[1,1,1]]))

(q^2/(-q^3*t^4+3*q^2*t^5-3*q*t^6+t^7+2*q^4*t^2-6*q^3*t^3+6*q^2*t^4-2*q*t^5-q^5+3*q^4*t-3*q^3*t^2+q^2*t^3))*McdHt[1, 1, 1] # McdHt[1, 1] # McdHt[1, 1, 1] + ((-q^3-q^2*t-q^2)/(-q^4*t^4+2*q^3*t^5-q^2*t^6+2*q^5*t^2-4*q^4*t^3+2*q^3*t^4+q^2*t^5-2*q*t^6+t^7-q^6+2*q^5*t-q^4*t^2-2*q^3*t^3+4*q^2*t^4-2*q*t^5+q^4*t-2*q^3*t^2+q^2*t^3))*McdHt[1, 1, 1] # McdHt[1, 1] # McdHt[2, 1] + (q^2/(q^5*t^2-3*q^4*t^3+3*q^3*t^4-q^2*t^5-q^6+3*q^5*t-3*q^4*t^2+3*q^2*t^4-3*q*t^5+t^6+q^4*t-3*q^3*t^2+3*q^2*t^3-q*t^4))*McdHt[1, 1, 1] # McdHt[1, 1] # McdHt[3] + ((-q*t)/(-q^3*t^4+3*q^2*t^5-3*q*t^6+t^7+2*q^4*t^2-6*q^3*t^3+6*q^2*t^4-2*q*t^5-q^5+3*q^4*t-3*q^3*t^2+q^2*t^3))*McdHt[1, 1, 1] # McdHt[2] # McdHt[1, 1, 1] + ((q^2*t+q*t^2+q*t)/(-q^4*t^4+2*q^3*t^5-q^2*t^6+2*q^5*t^2-4*q^4*t^3+2*q^3*t^4+q^2*t^5-2*q*t^6+t^7-q^6+2*q^5*t-q^4*t^2-2*q^3*t^3+4*q^2*t^4-2*q*t^5+q^4*t-2*q^3*t^2+q^2*t^3))*McdHt[1, 1, 1] # McdHt[2] # McdHt[2, 1] + ((-q*t)/(q^5*t^2-3*q^4*t^3+3*q^3*t^4-q^2*t^5-q^6+3*q^5*t-3*q^4*t^2+3*q^2*t^4-3*q*t^5+t^6+q^4*t-3*q^3

# Computations

In [96]:
A=wH([4,3,1,1,1],2)

In [25]:
wH([4,3,1,1,1],3)

t^3*s[] # s[] # s[1, 1, 1] + ((t^2+q)/q)*s[] # s[] # s[2, 1] - (1/(-q*t))*s[] # s[] # s[3] + ((2*q^2*t^2+t^3)/q^2)*s[] # s[1] # s[1, 1] + ((-t^4-q*t^2-q^2)/(-q^2*t))*s[] # s[1] # s[2] + ((t^5+q*t^3+q^2*t)/q^2)*s[] # s[1, 1] # s[1] + ((-t^5)/(-q))*s[] # s[1, 1, 1] # s[] + ((-q^2*t-2*t^2)/(-q^2))*s[] # s[2] # s[1] + ((t^4+q*t^2)/q^2)*s[] # s[2, 1] # s[] + ((-t)/(-q^2))*s[] # s[3] # s[] + ((q^2*t^2+t^3+q*t)/q)*s[1] # s[] # s[1, 1] + ((-t^4-q^2-t)/(-q*t))*s[1] # s[] # s[2] + ((-q^2*t^3-q^3*t-t^4-2*q*t^2-q^2)/(-q^2))*s[1] # s[1] # s[1] + ((-2*t^4-q*t^2)/(-q))*s[1] # s[1, 1] # s[] + ((q^2*t^2+t^3+q*t)/q^2)*s[1] # s[2] # s[] + ((q^2*t^3+t^4+q^2)/q)*s[1, 1] # s[] # s[1] + ((q*t^4+q^2*t^2+t^3)/q)*s[1, 1] # s[1] # s[] + q*t^4*s[1, 1, 1] # s[] # s[] + ((q*t^2+q^2+t)/q)*s[2] # s[] # s[1] + ((t^3+2*q*t)/q)*s[2] # s[1] # s[] + (t^3+q*t)*s[2, 1] # s[] # s[] + s[3] # s[] # s[]

In [26]:
wH([3,2,1],3)

q*t^2*s[] # s[] # s[1, 1] + t*s[] # s[] # s[2] + 2*q*t*s[] # s[1] # s[1] + q^2*t*s[] # s[1, 1] # s[] + q*s[] # s[2] # s[] + (t^2+q)*s[1] # s[] # s[1] + (q^2+t)*s[1] # s[1] # s[] + q*t*s[1, 1] # s[] # s[] + s[2] # s[] # s[]

In [27]:
wH([4,2,1],3)

t^3/q^4*s[] # s[] # s[1, 1] + ((-t)/(-q^3))*s[] # s[] # s[2] + ((-q^2*t-t^2)/(-q^4))*s[] # s[1] # s[1] + t^2/q^3*s[] # s[1, 1] # s[] - (1/(-q^2))*s[] # s[2] # s[] + ((-q^2*t-t^2)/(-q^3))*s[1] # s[] # s[1] + ((-t^2-q)/(-q^2))*s[1] # s[1] # s[] + ((-t^2)/(-q))*s[1, 1] # s[] # s[] + s[2] # s[] # s[]

In [6]:
wH([7,4,2,1],3)

t^2*s[] # s[] # s[1, 1, 1, 1] + ((q^4+q^2*t+t^2)/q^3)*s[] # s[] # s[2, 1, 1] + ((-q*t-1)/(-q^2))*s[] # s[] # s[2, 2] + ((-q^4-q^2*t-t^2)/(-q^4*t))*s[] # s[] # s[3, 1] + 1/(q^3*t)*s[] # s[] # s[4] + ((q^3*t^2+q^4+q^2*t+t^2)/q^2)*s[] # s[1] # s[1, 1, 1] + ((q^5*t+2*q^3*t^2+q^4+q*t^3+2*q^2*t+t^2)/(q^3*t))*s[] # s[1] # s[2, 1] + ((q^4+q^2*t+t^2+q)/(q^3*t))*s[] # s[1] # s[3] + ((-q^3*t^2-q^4-2*q^2*t-t^2-q)/(-q))*s[] # s[1, 1] # s[1, 1] + ((q^5*t+q^3*t^2+q^4+q*t^3+q^2*t+t^2)/(q^2*t))*s[] # s[1, 1] # s[2] + (q^3*t^2+q^4+q^2*t+t^2)*s[] # s[1, 1, 1] # s[1] + q^4*t^2*s[] # s[1, 1, 1, 1] # s[] + ((-q^5*t-q^3*t^2-q^4-q*t^3-q^2*t-t^2)/(-q^2*t))*s[] # s[2] # s[1, 1] + ((q^3*t^2+q^4+2*q^2*t+t^2+q)/(q^2*t))*s[] # s[2] # s[2] + ((q^5*t+2*q^3*t^2+q^4+q*t^3+2*q^2*t+t^2)/(q*t))*s[] # s[2, 1] # s[1] + (q^5+q^3*t+q*t^2)*s[] # s[2, 1, 1] # s[] + (q^3*t+q^2)*s[] # s[2, 2] # s[] + ((-q^4-q^2*t-t^2-q)/(-q*t))*s[] # s[3] # s[1] + ((q^4+q^2*t+t^2)/t)*s[] # s[3, 1] # s[] + ((-q)/(-t))*s[] # s[4] # s[] + ((2*q^2*t+

In [28]:
wH([7,6,6,1],4)

q^11*t^6*s[] # s[] # s[] # s[1, 1, 1, 1, 1] + (q^10*t^5+q^9*t^4+q^7*t^6+q^6*t^5)*s[] # s[] # s[] # s[2, 1, 1, 1] + (q^9*t^4+q^8*t^3+q^6*t^5+q^5*t^4+q^3*t^6)*s[] # s[] # s[] # s[2, 2, 1] + (q^8*t^3+q^6*t^5+2*q^5*t^4+q^4*t^3+q^2*t^5)*s[] # s[] # s[] # s[3, 1, 1] + (q^7*t^2+q^5*t^4+q^4*t^3+q^2*t^5+q*t^4)*s[] # s[] # s[] # s[3, 2] + (q^4*t^3+q^3*t^2+q*t^4+t^3)*s[] # s[] # s[] # s[4, 1] + t^2/q*s[] # s[] # s[] # s[5] + (2*q^11*t^5+q^10*t^4+q^8*t^6+q^7*t^5)*s[] # s[] # s[1] # s[1, 1, 1, 1] + (2*q^10*t^4+3*q^9*t^3+4*q^7*t^5+3*q^6*t^4+q^4*t^6+q^5*t^3+q^3*t^5)*s[] # s[] # s[1] # s[2, 1, 1] + (2*q^9*t^3+q^8*t^2+3*q^6*t^4+q^4*t^6+q^5*t^3+q^3*t^5+q^2*t^4)*s[] # s[] # s[1] # s[2, 2] + (2*q^8*t^2+3*q^6*t^4+4*q^5*t^3+2*q^3*t^5+q^4*t^2+2*q^2*t^4+q*t^3)*s[] # s[] # s[1] # s[3, 1] + (2*q^4*t^2+q^2*t^4+q*t^3+t^2)*s[] # s[] # s[1] # s[4] + (q^12*t^5+2*q^11*t^4+2*q^10*t^3+3*q^8*t^5+q^7*t^4+q^5*t^6)*s[] # s[] # s[1, 1] # s[1, 1, 1] + (q^11*t^4+3*q^10*t^3+2*q^8*t^5+2*q^9*t^2+5*q^7*t^4+q^5*t^6+3*q^6*t^3+2*q^4

In [54]:
wH([7,7,6,6,1],3)

q*t^10*s[] # s[] # s[1, 1, 1, 1, 1, 1] + ((-q^4*t^8-q^2*t^9-q^3*t^7-t^10-q*t^8)/(-q^2))*s[] # s[] # s[2, 1, 1, 1, 1] + ((q^8*t^6+q^6*t^7+q^7*t^5+2*q^4*t^8+q^5*t^6+q^2*t^9+q^3*t^7+t^10)/q^5)*s[] # s[] # s[2, 2, 1, 1] + ((q^8*t^4+q^4*t^6+q*t^9+q^2*t^7+t^8)/q^4)*s[] # s[] # s[2, 2, 2] + ((-q^5*t^7-q^6*t^5-q^3*t^8-2*q^4*t^6-q*t^9-2*q^2*t^7-q^3*t^5-t^8)/(-q^4))*s[] # s[] # s[3, 1, 1, 1] + ((-q^8*t^5-q^9*t^3-2*q^6*t^6-2*q^7*t^4-2*q^4*t^7-2*q^5*t^5-2*q^2*t^8-2*q^3*t^6-t^9-q*t^7)/(-q^6))*s[] # s[] # s[3, 2, 1] + ((-q^8*t^4-q^6*t^5-q^7*t^3-q^4*t^6-t^8)/(-q^7))*s[] # s[] # s[3, 3] + ((q^6*t^4+q^3*t^7+2*q^4*t^5+q^5*t^3+2*q^2*t^6+q^3*t^4+t^7+q*t^5)/q^5)*s[] # s[] # s[4, 1, 1] + ((-q^8*t^2-q^5*t^5-q^6*t^3-q^3*t^6-2*q^4*t^4-q*t^7-q^2*t^5-t^6)/(-q^6))*s[] # s[] # s[4, 2] + ((q^3*t^4+q^4*t^2+q*t^5+q^2*t^3+t^4)/q^5)*s[] # s[] # s[5, 1] + t^2/q^4*s[] # s[] # s[6] + ((-q^3*t^10-q^4*t^8-q^2*t^9-q^3*t^7-t^10-q*t^8)/(-q))*s[] # s[1] # s[1, 1, 1, 1, 1] + ((q^7*t^8+q^8*t^6+q^5*t^9+3*q^6*t^7+q^3*t^10+2*q^7*t^5

In [31]:
s_to_p(wH([5,5,2],3))

((1/24*q^8*t^3+1/8*q^7*t^2+1/8*q^5*t^3+1/12*q^6*t+1/4*q^4*t^2+1/12*q^2*t^3+1/8*q^3*t+1/8*q*t^2+1/24*t)/q)*p[] # p[] # p[1, 1, 1, 1] + ((-1/4*q^8*t^3-1/4*q^7*t^2-1/4*q^5*t^3+1/4*q^3*t+1/4*q*t^2+1/4*t)/q)*p[] # p[] # p[2, 1, 1] + ((1/8*q^8*t^3-1/8*q^7*t^2-1/8*q^5*t^3+1/4*q^6*t-1/4*q^4*t^2+1/4*q^2*t^3-1/8*q^3*t-1/8*q*t^2+1/8*t)/q)*p[] # p[] # p[2, 2] + ((1/3*q^8*t^3-1/3*q^6*t-1/3*q^2*t^3+1/3*t)/q)*p[] # p[] # p[3, 1] + ((-1/4*q^8*t^3+1/4*q^7*t^2+1/4*q^5*t^3-1/4*q^3*t-1/4*q*t^2+1/4*t)/q)*p[] # p[] # p[4] + ((-1/6*q^8*t^2-1/6*q^6*t^3-1/3*q^7*t-1/6*q^4*t^4-5/6*q^5*t^2-2/3*q^3*t^3-1/2*q^4*t-1/3*q*t^4-1/2*q^2*t^2-1/6*t^3-1/6*q*t)/(-q))*p[] # p[1] # p[1, 1, 1] + ((1/2*q^8*t^2+1/2*q^6*t^3+1/2*q^4*t^4+1/2*q^5*t^2-1/2*q^4*t-1/2*q^2*t^2-1/2*t^3-1/2*q*t)/(-q))*p[] # p[1] # p[2, 1] + ((-1/3*q^8*t^2-1/3*q^6*t^3+1/3*q^7*t-1/3*q^4*t^4+1/3*q^5*t^2+2/3*q^3*t^3+1/3*q*t^4-1/3*t^3-1/3*q*t)/(-q))*p[] # p[1] # p[3] + ((-1/4*q^9*t-1/4*q^6*t^4-3/4*q^7*t^2-q^5*t^3-3/4*q^6*t-3/4*q^3*t^4-q^4*t^2-3/4*q^2*t^3-1/4*q^3

# Old checks

In [1131]:
cells_between_mu_la(Partition([4,2,2]),Partition([1,1]))

[(0, 1), (0, 2), (0, 3), (1, 1), (2, 0), (2, 1)]

In [1032]:
color_i_addable_cells(Partition([4,1]),2,3)

[(0, 4), (2, 0)]

In [1073]:
wH41_3_0 = wH(Partition([4,1]),3,1)
wH41_3_0.terms()

[s[1] # s[] # s[], 1/q*s[] # s[1] # s[], 1/q^2*s[] # s[] # s[1]]

In [1040]:
cmula_1(Partition([4,1]), Partition([1,1]), 1, 3)

(-q^5 + q^3*t + q^2 - t)/q^3

In [ ]:
wH41_3_0 = wH(Partition([4,1]),3,1)
wH41_3_0.terms()

In [1123]:
wH422_3_0 = wH(Partition([4,2,2]),3,0)
wH422_3_1 = wH(Partition([4,2,2]),3,1)
wH422_3_2 = wH(Partition([4,2,2]),3,2)

wH422_3_0 - wH422_3_1

0

In [1030]:
s_to_m(wH422_3_1)

((q^8*t^2-q^5*t^5-q^9+q^3*t^6+q^7*t-q^4*t^4-q^5*t^2+q^2*t^5+q^6-t^6-q^4*t+q*t^4)/(-q^7))*m[] # m[] # m[1, 1] + ((q^8*t^2-q^5*t^5-q^9+q^3*t^6+q^7*t-q^4*t^4-q^5*t^2+q^2*t^5+q^6-t^6-q^4*t+q*t^4)/(-q^6*t^2-q^7))*m[] # m[] # m[2] + ((q^8*t^2-q^5*t^5-q^9+q^3*t^6+q^7*t-q^4*t^4-q^5*t^2+q^2*t^5+q^6-t^6-q^4*t+q*t^4)/(-q^6))*m[] # m[1] # m[1] + ((q^8*t^2-q^5*t^5-q^9+q^3*t^6+q^7*t-q^4*t^4-q^5*t^2+q^2*t^5+q^6-t^6-q^4*t+q*t^4)/(-q^5))*m[] # m[1, 1] # m[] + ((q^8*t^2-q^5*t^5-q^9+q^3*t^6+q^7*t-q^4*t^4-q^5*t^2+q^2*t^5+q^6-t^6-q^4*t+q*t^4)/(-q^4*t^2-q^5))*m[] # m[2] # m[] + ((q^8*t^2-q^5*t^5-q^9+q^3*t^6+q^7*t-q^4*t^4-q^5*t^2+q^2*t^5+q^6-t^6-q^4*t+q*t^4)/(-q^6*t))*m[1] # m[] # m[1] + ((q^8*t^2-q^5*t^5-q^9+q^3*t^6+q^7*t-q^4*t^4-q^5*t^2+q^2*t^5+q^6-t^6-q^4*t+q*t^4)/(-q^5*t))*m[1] # m[1] # m[] + ((q^8*t^2-q^5*t^5-q^9+q^3*t^6+q^7*t-q^4*t^4-q^5*t^2+q^2*t^5+q^6-t^6-q^4*t+q*t^4)/(-q^4*t))*m[1, 1] # m[] # m[] + ((q^8*t^2-q^5*t^5-q^9+q^3*t^6+q^7*t-q^4*t^4-q^5*t^2+q^2*t^5+q^6-t^6-q^4*t+q*t^4)/(-q^3*t^3-q^4*t))*m[2

In [1078]:
wH5422_4_0 = wH(Partition([5,4,2,2]),4,0)
wH5422_4_1 = wH(Partition([5,4,2,2]),4,1)
wH5422_4_2 = wH(Partition([5,4,2,2]),4,2)
wH5422_4_3 = wH(Partition([5,4,2,2]),4,3)

In [1086]:
wH5422_4_0 

t^2*s[] # s[] # s[] # s[1, 1] + ((-t^3)/(-q^3))*s[] # s[] # s[] # s[2] + ((-q^3*t^2-t^3)/(-q^2))*s[] # s[] # s[1] # s[1] + q^2*t^2*s[] # s[] # s[1, 1] # s[] + t^3/q*s[] # s[] # s[2] # s[] + ((-q^3*t-t^2)/(-q^2))*s[] # s[1] # s[] # s[1] + ((-q^3*t-t^2)/(-q))*s[] # s[1] # s[1] # s[] + q^2*s[] # s[1, 1] # s[] # s[] + t/q*s[] # s[2] # s[] # s[] + ((-q^2-t^2)/(-q))*s[1] # s[] # s[] # s[1] + (q^2+t^2)*s[1] # s[] # s[1] # s[] + ((-q^2-t^2)/(-t))*s[1] # s[1] # s[] # s[] + ((-q^3)/(-t))*s[1, 1] # s[] # s[] # s[] + s[2] # s[] # s[] # s[]

In [1011]:
 wH5422_4_3

((q^16*t^5-2*q^14*t^7+q^12*t^9-q^15*t^4+2*q^11*t^8-q^7*t^12+2*q^12*t^5-2*q^10*t^7-2*q^6*t^11+2*q^4*t^13-q^9*t^6+2*q^5*t^10-q*t^14+q^4*t^9-2*q^2*t^11+t^13)/q^5)*s[] # s[] # s[] # s[1, 1] + ((-q^16*t^3+2*q^14*t^5-q^12*t^7+q^15*t^2-2*q^11*t^6+q^7*t^10-2*q^12*t^3+2*q^10*t^5+2*q^6*t^9-2*q^4*t^11+q^9*t^4-2*q^5*t^8+q*t^12-q^4*t^7+2*q^2*t^9-t^11)/(-q^3))*s[] # s[] # s[] # s[2] + ((-q^18*t^3+q^16*t^5+q^14*t^7-q^12*t^9+q^17*t^2+q^15*t^4-2*q^13*t^6-2*q^11*t^8+q^9*t^10+q^7*t^12-2*q^14*t^3+2*q^10*t^7+2*q^8*t^9-2*q^4*t^13+q^11*t^4+q^9*t^6-2*q^7*t^8-2*q^5*t^10+q^3*t^12+q*t^14-q^6*t^7+q^4*t^9+q^2*t^11-t^13)/(-q^4))*s[] # s[] # s[1] # s[1] + ((q^16*t^5-2*q^14*t^7+q^12*t^9-q^15*t^4+2*q^11*t^8-q^7*t^12+2*q^12*t^5-2*q^10*t^7-2*q^6*t^11+2*q^4*t^13-q^9*t^6+2*q^5*t^10-q*t^14+q^4*t^9-2*q^2*t^11+t^13)/q^3)*s[] # s[] # s[1, 1] # s[] + ((-q^16*t^3+2*q^14*t^5-q^12*t^7+q^15*t^2-2*q^11*t^6+q^7*t^10-2*q^12*t^3+2*q^10*t^5+2*q^6*t^9-2*q^4*t^11+q^9*t^4-2*q^5*t^8+q*t^12-q^4*t^7+2*q^2*t^9-t^11)/(-q))*s[] # s[] # s[2] # s

In [1088]:
wH6444222_4_0 = wH(Partition([6,4,4,4,2,2,2]),4,0)
wH6444222_4_1 = wH(Partition([6,4,4,4,2,2,2]),4,1)
wH6444222_4_2 = wH(Partition([6,4,4,4,2,2,2]),4,2)
wH6444222_4_3 = wH(Partition([6,4,4,4,2,2,2]),4,3)

In [1115]:
cmula(Partition([4,2,2]), Partition([1,1]), 2, 0, 3)

(-t)/(-q^3)

In [1116]:
cmula(Partition([4,2,2]), Partition([1,1]), 2, 1, 3)

(-t)/(-q^3)

In [1117]:
cmula(Partition([4,2,2]), Partition([1,1]), 2, 2, 3)

(-t)/(-q^3)

In [1098]:
A0 = cmula(Partition([6,4,4,4,2,2,2]), Partition([5, 3, 2, 1, 1]), 2, 0, 4)
A1 = cmula(Partition([6,4,4,4,2,2,2]), Partition([5, 3, 2, 1, 1]), 2, 1, 4)
A2 = cmula(Partition([6,4,4,4,2,2,2]), Partition([5, 3, 2, 1, 1]), 2, 2, 4)
A3 = cmula(Partition([6,4,4,4,2,2,2]), Partition([5, 3, 2, 1, 1]), 2, 3, 4)

In [1099]:
A0

(-q^2*t^12 - q^5*t^7 + 2*q*t^11 - q^6*t^4 + 3*q^4*t^6 + q^2*t^8 - t^10 + 2*q^5*t^3 - 2*q^3*t^5 - 2*q*t^7 - q^4*t^2 - q^2*t^4 + t^6 + q*t^3)/(q^8 - 2*q^6*t^2 + q^4*t^4 - q^5*t + 2*q^3*t^3 - q*t^5)

In [1112]:
cmula_1(Partition([6,4,4,4,2,2,2]), Partition([6, 4, 3, 3, 2, 2]), 2, 4)

(q*t^19 - q^6*t^12 - t^18 + q^5*t^11 - q^3*t^13 + q^8*t^6 + q^2*t^12 - q^7*t^5)/(-q^9*t^5 + 2*q^7*t^7 - q^5*t^9 + q^12 - 2*q^10*t^2 + q^8*t^4)

In [1114]:
cmula_1(Partition([4,2,2]), Partition([3,2]), 2, 4)

(q^4*t^6 - q^5*t^3 - q^3*t^5 + q^4*t^2)/(-q^2 + t^2)

In [1113]:
k_predecessors(Partition([4,2,2]),1,3)

[[4, 1], [3, 2]]

In [1095]:
k_predecessors(Partition([6,4,4,4,2,2,2]),3,4)

[[6, 3, 3],
 [6, 2, 2, 1, 1],
 [6, 2, 1, 1, 1, 1],
 [5, 3, 3, 1],
 [5, 3, 2, 1, 1],
 [5, 3, 1, 1, 1, 1],
 [4, 3, 3, 1, 1],
 [3, 3, 3, 1, 1, 1],
 [2, 2, 2, 2, 2, 2]]

In [840]:
SkewPartition([Partition([4,2,2]), Partition([1,1])]).cells()

[(0, 1), (0, 2), (0, 3), (1, 1), (2, 0), (2, 1)]

In [844]:
chars_of_cells_between_mu_la(Partition([4,2,2]), Partition([1,1]))

[q, q^2, q^3, q*t, t^2, q*t^2]

In [809]:
multipartition_to_monomial(A)

-3*s[1, 1, 1, 1] # s[] # s[1] + s[2, 1, 1] # s[] # s[1]

In [777]:
M_mu_Gamma(Partition([4,4,3,2,2]), [Partition([2]),Partition([]),Partition([])], 2, 3)

[[0, 2]]
[[[4, 4, 3, 2, 2], [4, 2, 2, 1]], [[4, 4, 3, 2, 2], [4, 2, 1, 1, 1]]]


(2*t^2 + 3*q)/(t^2 + q)

In [931]:
seq = [6,5,5,3,2,1,1]
res = 3
P1 = Partition(seq)
alf = P1.core(res)
print(core_to_cees(P1,res))
print(core_to_cees(alf,res))

[1, 1, -2]
[1, 1, -2]


In [1125]:
char_of_cell((2,4))

q^4*t^2

In [ ]:
core_to_cees(Partition([4,4,3,2]),3)

In [764]:
prod_of_cmula([Partition([4,4,3,2,2]), Partition([4,2])],[1],0,3)

[1]


1

In [883]:
core_to_cees(Partition([5,5,3,3,2]),3)

[3, 0, -3]

In [884]:
core_to_cees(Partition([4,2]),3)

[3, 0, -3]

In [885]:
core_to_cees(Partition([1,1]),3)

[1, 1, -2]

In [766]:
prod_of_cmula([Partition([4,4,3,2,2]), Partition([4,2])],[1],1,3)

[(q^6*t + 4*q^3*t^4 + 2*t^7 + q^4*t^2 + 2*q*t^5)/(q^8 + 2*q^5*t^3 + q^2*t^6 + 2*q^6*t + 3*q^3*t^4 + t^7 + q^4*t^2 + q*t^5)]


(q^6*t + 4*q^3*t^4 + 2*t^7 + q^4*t^2 + 2*q*t^5)/(q^8 + 2*q^5*t^3 + q^2*t^6 + 2*q^6*t + 3*q^3*t^4 + t^7 + q^4*t^2 + q*t^5)

In [753]:
P1 = Partition([4,4,3,2,2])
P2 = Partition([4,2])
al = P1.core(3)

In [754]:
cmula(P1, P2, 1, 1, 3)

(q^6*t + 4*q^3*t^4 + 2*t^7 + q^4*t^2 + 2*q*t^5)/(q^8 + 2*q^5*t^3 + q^2*t^6 + 2*q^6*t + 3*q^3*t^4 + t^7 + q^4*t^2 + q*t^5)

In [557]:
cmula(P1, P2, 2, 1, 3)

(q^6*t + 4*q^3*t^4 + 2*t^7 + q^4*t^2 + 2*q*t^5)/(q^8 + 2*q^5*t^3 + q^2*t^6 + 2*q^6*t + 3*q^3*t^4 + t^7 + q^4*t^2 + q*t^5)

In [553]:
B_mu_la_s(P1,P2,0,3)

q*t^4 + q^2*t^2 + t^3

In [563]:
Tmu_nu_s(Partition([4,4,3,2,2]), Partition([4,2]), 1, 3)

-q^4*t^7

In [624]:
P1 = Partition([6,2,1])
P2 = Partition([4,2,2])
al = P1.core(4)

In [628]:
cmula(P1, al, 0, 2, 4)

(2*q^2 + t^2)/(q^2 + t^2)

In [657]:
multipartition_to_sequence([[2,2],[1]],2)

[[0, 2], [0, 2], [1, 1]]

In [623]:
k_l_predecessors(P1, al,1, 3)

[[6, 1], [3, 2, 2]]

In [523]:
cmula(mu, la, p, es, r)

[4, 2]

In [569]:
k_l_predecessors(Partition([2,1,1,1,1,1,1]), Partition([2]),1, 3)

[[2, 1, 1, 1]]

In [330]:
color_s_cells_mu_la(Partition([4,2,2]),Partition([1,1]), 0, 3)

[(0, 3), (1, 1)]

In [331]:
noncore_color_s_cells(Partition([4,2,2]), 0, 3)

[(0, 3), (1, 1)]

In [338]:
cells_to_characters([(0, 0), (0, 3), (1, 1)])

[1, q^3, q*t]

In [342]:
cells_to_neg_characters([(0, 0), (0, 3), (1, 1)])

[-1, -q^3, -q*t]

In [439]:
Partitions(-1).list()

[]

In [110]:
Partition([6,3,2,2,2]).quotient(3)

([], [1, 1], [2, 1])

In [47]:
Partition(core=[2,1], quotient=[[2,1],[3],[1,1,1]])

[11, 5, 5, 3, 2, 2, 2]

In [65]:
Partition([3,2,1]).corners_residue(1, 3)

[(0, 2)]

In [50]:
Partition([2,2,1]).remove_cell(2)

[2, 2]

In [64]:
 Partition([2,1]).residue(1, 0, 3)

1

In [74]:
Partitions(4, outer=Partition([5,1,1])).list()

[[4], [3, 1], [2, 1, 1]]

In [636]:
[[2,2],[1]][0]

[2, 2]

In [258]:
noncore_color_s_cells(Partition([4,2,2]), 0, 3)

[(0, 1), (0, 2), (0, 3), (1, 1), (2, 0), (2, 1)]

In [230]:
Partition([5, 4, 3, 1, 1]).core(3)

[1, 1]

In [203]:
Partition([6,4,2,1,1]).core(3)

[6, 4, 2, 1, 1]

In [135]:
Partition([6,4,3,2,2]).core(3)

[1, 1]

In [231]:
list1 = [1,2,3,4,5,6,7,8,9,10]
for i in list1:
    if mod(i,2)!=0:
        list1.remove(i)
list1
    

[2, 4, 6, 8, 10]

In [232]:
list1 = [1,2,3]
list1.remove(2)
list1

[1, 3]

In [344]:
@cached_function
def B_mu_la_s(mu, la, s, r):
    r"""
    Return the sum of (q^a t^b), where (a,b) is a cell in mu/la of color s.

    INPUT:

    - ``mu``, ``la``, ``s``, ``r`` -- partition ``mu``, partition ``la``, color ``s``, residue ``r``

    OUTPUT: an element of the fraction field of polynomials in `q` and `t`

    EXAMPLES::
        sage:B_mu_la_s(Partition([4,2,2]),Partition([1,1]),0,3)
             q^3 + q*t
    """
    cells_to_add = color_s_cells_mu_la(mu,la, s, r)
    cell_characters = cells_to_characters(cells_to_add)
    return sum(cell_characters)


In [370]:
B_mu_la_s(Partition([4,2,2]),Partition([1,1]),0,3)

q^3 + q*t

In [371]:
s[2](B_mu_la_s(Partition([4,2,2]),Partition([1,1]),0,3)*s[0]).scalar(s[0])

q^6 + q^4*t + q^2*t^2

In [320]:
Tmu_s(Partition([4,2,2]), 1, 3)

q^3*t^2

In [307]:
list1 = noncore_color_s_cells(Partition([4,2,2]),0,3)
list1

[(0, 1), (0, 2), (0, 3), (1, 1), (2, 0), (2, 1)]

In [294]:
[t^(c[0]) for c in list1]

[1, 1, 1, t, t^2, t^2]

In [313]:
residue(2,0,2)

TypeError: residue() missing 1 required positional argument: 'l'

In [366]:
s[3](q*s[0]).scalar(s[0])

q^3

In [1067]:
cmula(Partition([4,1]), Partition([1,1]), 0, 0, 3)

(-q^5 + q^3*t + q^2 - t)/q^2

In [1059]:
A = Matrix([[1,1,1],[q^(-1),q*t^(-1),t],[q^(-2),t^(-1),t^(-1)]])

In [1060]:
A

[    1     1     1]
[  1/q   q/t     t]
[1/q^2   1/t   1/t]

In [1062]:
X = A.solve_right(vector([1+q^2 *t^2 + q*t, q^2 + t + q*t^2,q+q^2*t+t^2]))

In [1064]:
X.list()

[(q^2*t^3 - q^2)/(-q^2 + t),
 (-q^4*t^4 + q^4*t + q*t^4 - q*t)/(-q^2*t^2 + q^3 + t^3 - q*t),
 (q^3*t^2 - t^2)/(-t^2 + q)]

In [18]:
s_to_p(wH(Partition([6,5,4]),4)).terms()

[((-1/3*q^3*t-1/6*q*t^3-1/6*q^2-1/3*t^2)/(-q^2))*p[1, 1, 1] # p[] # p[] # p[],
 ((-1/2*t^3+1/2*q)/q)*p[2, 1] # p[] # p[] # p[],
 ((1/3*q^3*t-1/3*q*t^3-1/3*q^2+1/3*t^2)/(-q^2))*p[3] # p[] # p[] # p[],
 ((-1/2*q^6-1/2*q^4*t^2-q^3*t-1/2*q*t^3-1/2*t^2)/(-q^3))*p[1, 1] # p[1] # p[] # p[],
 ((-1/2*q^6+1/2*q^4*t^2+1/2*q*t^3-1/2*t^2)/(-q^3))*p[2] # p[1] # p[] # p[],
 ((-1/2*q^6-1/2*q^4*t^2-q^3*t-1/2*q*t^3-1/2*t^2)/(-q^4))*p[1, 1] # p[] # p[1] # p[],
 ((-1/2*q^6+1/2*q^4*t^2+1/2*q*t^3-1/2*t^2)/(-q^4))*p[2] # p[] # p[1] # p[],
 ((-q^3*t-1/2*q*t^3-1/2*q^2-t^2)/(-q))*p[1, 1] # p[] # p[] # p[1],
 (-1/2*t^3+1/2*q)*p[2] # p[] # p[] # p[1],
 ((-1/2*q^6*t-1/2*q^5-q^3*t^2-1/2*q^2*t-1/2*t^3)/(-q^3))*p[1] # p[1, 1] # p[] # p[],
 ((1/2*q^6*t-1/2*q^5-1/2*q^2*t+1/2*t^3)/(-q^3))*p[1] # p[2] # p[] # p[],
 ((-q^6*t-q^5-2*q^3*t^2-q^2*t-t^3)/(-q^4))*p[1] # p[1] # p[1] # p[],
 ((-q^6-q^4*t^2-2*q^3*t-q*t^3-t^2)/(-q^2))*p[1] # p[1] # p[] # p[1],
 ((-1/2*q^6*t-1/2*q^5-q^3*t^2-1/2*q^2*t-1/2*t^3)/(-q^5))*p[1] # p[] # p[

In [1105]:
s_to_p(wH52_0)

((-1/2*q^4-1/2*q)/(-t))*p[] # p[] # p[1, 1] + ((1/2*q^4-1/2*q)/(-t))*p[] # p[] # p[2] + ((-q^2-t)/(-t))*p[] # p[1] # p[1] + ((-1/2*q^3-1/2)/(-q^2))*p[] # p[1, 1] # p[] + ((1/2*q^3-1/2)/(-q^2))*p[] # p[2] # p[] + ((-q^3-q*t)/(-t))*p[1] # p[] # p[1] + ((-q^3-1)/(-q))*p[1] # p[1] # p[] + (1/2*q^3+1/2)*p[1, 1] # p[] # p[] + (-1/2*q^3+1/2)*p[2] # p[] # p[]

In [1107]:
Partition([5,3,3,2]).core(4)

[1]

In [1108]:
wH52_0 = wH(Partition([5,3,3,2]),4,0)
s_to_p(wH52_0)

((1/6*q*t^5+1/3*q^2*t^2+1/3*t^4+1/6*q*t)/q^5)*p[] # p[] # p[] # p[1, 1, 1] + ((-1/2*t^5+1/2*t)/q^4)*p[] # p[] # p[] # p[2, 1] + ((1/3*q*t^5-1/3*q^2*t^2-1/3*t^4+1/3*q*t)/q^5)*p[] # p[] # p[] # p[3] + ((1/2*q^4*t^2+q^2*t^4+1/2*t^6+1/2*q^3*t+1/2*q*t^3)/q^6)*p[] # p[] # p[1] # p[1, 1] + ((-1/2*q^4*t^2-1/2*t^6+1/2*q^3*t+1/2*q*t^3)/q^6)*p[] # p[] # p[1] # p[2] + ((1/2*q^3*t^4+1/2*q*t^6+1/2*q^4*t+q^2*t^3+1/2*t^5)/q^6)*p[] # p[] # p[1, 1] # p[1] + ((1/6*q*t^6+1/3*q^2*t^3+1/3*t^5+1/6*q*t^2)/q^5)*p[] # p[] # p[1, 1, 1] # p[] + ((-1/2*q^3*t^4-1/2*q*t^6+1/2*q^4*t+1/2*t^5)/q^6)*p[] # p[] # p[2] # p[1] + ((-1/2*t^6+1/2*t^2)/q^4)*p[] # p[] # p[2, 1] # p[] + ((1/3*q*t^6-1/3*q^2*t^3-1/3*t^5+1/3*q*t^2)/q^5)*p[] # p[] # p[3] # p[] + ((1/2*q^5*t^2+1/2*q^4*t+q^2*t^3+1/2*t^5+1/2*q*t^2)/q^6)*p[] # p[1] # p[] # p[1, 1] + ((-1/2*q^5*t^2+1/2*q^4*t-1/2*t^5+1/2*q*t^2)/q^6)*p[] # p[1] # p[] # p[2] + ((q^5*t+2*q^3*t^3+q*t^5+q^2*t^2+t^4)/q^6)*p[] # p[1] # p[1] # p[1] + ((1/2*q^3*t^3+1/2*q*t^5+q^2*t^2+t^4)/q^5)*p[] #

In [1130]:
wH(Partition([5,3,3,2]),4)

t^5/q^4*s[] # s[] # s[] # s[1, 1, 1] + ((q^2*t^2+t^4)/q^5)*s[] # s[] # s[] # s[2, 1] + t/q^4*s[] # s[] # s[] # s[3] + ((q^4*t^2+q^2*t^4+t^6)/q^6)*s[] # s[] # s[1] # s[1, 1] + ((q*t^4+q^2*t+t^3)/q^5)*s[] # s[] # s[1] # s[2] + ((q^2*t^4+t^6+q*t^3)/q^5)*s[] # s[] # s[1, 1] # s[1] + t^6/q^4*s[] # s[] # s[1, 1, 1] # s[] + ((q^4*t+q^2*t^3+t^5)/q^6)*s[] # s[] # s[2] # s[1] + ((q^2*t^3+t^5)/q^5)*s[] # s[] # s[2, 1] # s[] + t^2/q^4*s[] # s[] # s[3] # s[] + ((q^5*t^2+q^2*t^3+t^5)/q^6)*s[] # s[1] # s[] # s[1, 1] + ((q^3*t+q*t^3+t^2)/q^5)*s[] # s[1] # s[] # s[2] + ((q^5*t+2*q^3*t^3+q*t^5+q^2*t^2+t^4)/q^6)*s[] # s[1] # s[1] # s[1] + ((-q^3*t^3-q*t^5-t^4)/(-q^5))*s[] # s[1] # s[1, 1] # s[] + ((-2*q^2*t^2-t^4)/(-q^5))*s[] # s[1] # s[2] # s[] + ((q^3*t^3+q^2*t^2+t^4)/q^5)*s[] # s[1, 1] # s[] # s[1] + ((q^2*t^2+2*t^4)/q^4)*s[] # s[1, 1] # s[1] # s[] + t^4/q^3*s[] # s[1, 1, 1] # s[] # s[] + ((q^5+q^3*t^2+t^3)/q^6)*s[] # s[2] # s[] # s[1] + ((q^3*t^2+q^2*t+t^3)/q^5)*s[] # s[2] # s[1] # s[] + ((q^2*t+t^3)

In [11]:
wH(Partition([10,5,5,3,3,2]),5)

q^9*t^5*s[] # s[] # s[] # s[] # s[1, 1, 1, 1] + (q^8*t^4+q^10*t+q^4*t^5)*s[] # s[] # s[] # s[] # s[2, 1, 1] + (q^10*t+q^3*t^4)*s[] # s[] # s[] # s[] # s[2, 2] + (q^9+q^3*t^4+q^5*t)*s[] # s[] # s[] # s[] # s[3, 1] + q^4*s[] # s[] # s[] # s[] # s[4] + (2*q^9*t^4+q^11*t+q^5*t^5)*s[] # s[] # s[] # s[1] # s[1, 1, 1] + (2*q^11*t+q^8*t^3+q^10+3*q^4*t^4+q^6*t)*s[] # s[] # s[] # s[1] # s[2, 1] + (q^10+q^6*t+q^3*t^3+q^5)*s[] # s[] # s[] # s[1] # s[3] + (q^10*t^4+2*q^12*t+q^9*t^3+2*q^5*t^4)*s[] # s[] # s[] # s[1, 1] # s[1, 1] + (q^12*t+q^9*t^3+q^11+q^5*t^4+q^7*t+q^4*t^3)*s[] # s[] # s[] # s[1, 1] # s[2] + (q^11*t^4+q^13*t+q^10*t^3+q^6*t^4)*s[] # s[] # s[] # s[1, 1, 1] # s[1] + q^12*t^4*s[] # s[] # s[] # s[1, 1, 1, 1] # s[] + (q^9*t^3+2*q^11+2*q^5*t^4+q^7*t)*s[] # s[] # s[] # s[2] # s[1, 1] + (2*q^11+q^7*t+2*q^4*t^3+q^6)*s[] # s[] # s[] # s[2] # s[2] + (q^10*t^3+3*q^12+q^6*t^4+q^8*t+2*q^5*t^3)*s[] # s[] # s[] # s[2, 1] # s[1] + (q^11*t^3+q^13+q^7*t^4)*s[] # s[] # s[] # s[2, 1, 1] # s[] + (q^13+q^6